In [ ]:
#integration functions
import random
from skhep.math.vectors import LorentzVector, Vector3D
import math
self=model
def integrate(br, coupling, m0, m1, m2, m3, nsample):

    # prepare output
    particles, weights = [], []
    #create parent 4-vector
    p_mother=LorentzVector(0,0,0,m0)

    #integration boundary
    q2min,q2max = (m2+m3)**2,(m0-m1)**2
    mass = m3
    #numerical integration
    integral=0
    for i in range(nsample):

        #Get kinematic Variables
        q2 = random.uniform(q2min,q2max)
        q  = math.sqrt(q2)
        E2st = (q**2 - m2**2 + m3**2)/(2*q)
        E3st = (m0**2 - q**2 - m1**2)/(2*q)
        m232min = (E2st + E3st)**2 - (np.sqrt(E2st**2 - m3**2) + np.sqrt(E3st**2 - m1**2))**2
        m232max = (E2st + E3st)**2 - (np.sqrt(E2st**2 - m3**2) - np.sqrt(E3st**2 - m1**2))**2
        cthmax = (m232max + q**2 - m2**2 - m1**2)/(2*m0)
        cthmin = (m232min + q**2 - m2**2 - m1**2)/(2*m0)
        ENmax = (m232max + q**2 - m2**2 - m1**2)/(2*m0)
        ENmin = (m232min + q**2 - m2**2 - m1**2)/(2*m0)
        EN = random.uniform(ENmin,ENmax)
        #th = np.arccos(EN)


        #decay meson and V
        #cosQ =random.uniform(-1,1)
        #phiQ =random.uniform(-math.pi,math.pi)
        #cosM =random.uniform(-1.,1.)
        #phiM =random.uniform(-math.pi,math.pi)
        #branching fraction
        brval  = eval(br)
        #print("q val ",q," EN val ",EN," Br ",brval, " Br ",eval(br))
        brval *= (q2max-q2min)*(cthmax-cthmin)/float(nsample)

        #save
        #particles.append(p_3)
        weights.append(brval)
    #print('branching frac', brval)
    #return particles,weights
    return(sum(weights))

def integrate_q2EN(pid0,pid1,pid2,m3,coupling,dbr,nsample):
    m0=self.masses(f'{pid0}')
    m1=self.masses(f'{pid1}')
    m2=self.masses(f'{pid2}')
    integ=integrate(dbr, 1, m0, m1, m2,m3, nsample)
    return(integ)
    
def integrate_EN(br, coupling, m0, m1, m2, m3, nsample):
    #prepare output
    weights=[]
    mass = m3
    #numerical integration
    integral=0
    for i in range(nsample):
        #get energy bounds
        ENmin=m3
        ENmax=(m0**2+m3**2-m1**2)/(2*m0)
        EN = random.uniform(ENmin,ENmax)

        #branching fraction
        brval  = eval(br)
        brval *= (ENmax-ENmin)/float(nsample)

        #save
        weights.append(brval)
    return(sum(weights))


#pid0 is parent particle, pid1 is produced meson pid2 is the smaller particle
#
from matplotlib import pyplot as plt
self=model
def show_br_curve(br,pid0,pid1,pid2,integration,nsample=100):
    if integration=='br':
        coupling=1
        delm=.1
        mass0=0
        mass=0.01
        x=[]
        y=[]
        for n in range(1,int((3-mass0)/delm)):
            y.append(eval(br))
            x.append(mass)
            mass+=delm

        plt.plot(x,y)
        plt.xlim([0,3])
        plt.ylim([0,max(y)])
        plt.xlabel(r"$m_N (GeV)$")
        plt.ylabel(r"Br")

    if integration=='dEN':
        dbr=br
        m0=self.masses(f"{pid0}")
        m1=self.masses(f"{pid1}")
        m2=self.masses(f"{pid2}")
        m3=0
        qmin,qmax=(m2+m3),(m0-m1)
        coupling=1
        delm=.1
        x=[]
        y=[]
        for n in range(1,30):
            y.append(integrate_EN(dbr, 1, m0, m1, m2,m3, nsample))
            x.append(m3)
            m3+=delm
        plt.xlim([0,2])
        plt.ylim([0,max(y)])
        plt.xlabel(r"$m_N (GeV)$")
        plt.ylabel(r"Br")
        plt.plot(x,np.array(y))

    if integration=='dq2dEN':
        dbr=br
        m0=self.masses(f"{pid0}")
        m1=self.masses(f"{pid1}")
        m2=self.masses(f"{pid2}")
        coupling=1
        delm=.1
        x=[]
        y=[]
        m3=0
        for n in range(1,30):
            y.append(integrate_q2EN(pid0,pid1,pid2,m3,coupling,dbr,nsample))
            #y.append(integrate_pseud_3body(df,pid0,pid1,pid2,m3,1,nsample=nsample,channel="D"))
            x.append(m3)
            m3+=delm
        plt.ylim([0,max(y)])
        plt.xlabel(r"$m_N (GeV)$")
        plt.ylabel(r"Br")
        plt.plot(x,np.array(y))

    plt.show()


import pandas as pd

#n can be: 2 for 2-body and 3 for 3-body. type can be: 2 body regular, 2 body tau to pi, 2 body tau to K, 2 body tau to rho, 3 body pseudo, 3 body vector, 3 body tau
#pid0 is parent pid1 is produces meson pid2 is the smaller particle, usually the electron
#for 2 body pid0 is the parent and pid1 is the produced SM particle
def add_nbodydecay(pid0,pid1,pid2,energy='14',nsample=10,generator="Pythia8",n=2,type_decay="2 body regular",show_br=True):
    if n==2 and type_decay=="2 body regular":
        df=pd.DataFrame([],columns=['particle','pid0','tauH (sec)','MH (GeV)','VH (unitless)','fH (GeV)','process'])
        particles=[['D^+','411',1040*10**(-15),1.8696,.221,.2226,'D^+ -> e^+ + N'],['D^+_s','431',500*10**(-15),1.96847,0.987,.2801, 'D_s^+ -> e^+ + N'],['B^+','521',1.638*10**(-12),5.27934,3.82*10**(-3),0.187,'B^+ -> e^+ + N'],['B^+_c','541',0.510*10**(-12),6.2749,41*10**(-3),.434, 'B^+_c -> e^+ + N']]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]
        br=hnl.get_2body_br(pid0,pid1)
        model.add_production_2bodydecay(
        pid0 = f"{pid0}",
        pid1 = f"{pid1}",
        br=br,
        generator = generator,
        energy = energy,
        nsample = nsample
        )
        if show_br==True:
            show_br_curve(br,pid0,pid1=pid1,pid2=pid2,integration='br',nsample=nsample)
        #show_br_curve(df,br,pid0,pid1=None,pid2=pid2,decay_type='2 body',nsample=nsample)
        
    if n==2 and type_decay=="2 body tau":
        df=pd.DataFrame([],columns=['pid0','pid1','process'])
        particles=[['15','-211','tau^- -> pi^- + N'],['15','-321','tau^- -> K^- + N'],['15','-213','tau^- -> rho^- + N']]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]

        if pid1=="-213" or pid1=="213":
            br=hnl.get_2body_br_tau_rho(pid0,pid1)
        else:
            br=hnl.get_2body_br_tau_H(pid0,pid1)

        model.add_production_2bodydecay(
            pid0 = f"{pid0}",
            pid1 = f"{pid1}",
            br=br,
            generator = generator,
            energy = energy,
            nsample = nsample
        )
        if show_br==True:
            show_br_curve(br,pid0,pid1=pid1,pid2=pid2,integration='br',nsample=nsample)
        
    #here pid1 is the electron (lepton) now and pid2 is the neutrino
    if n==3 and type_decay=="3 body tau":
        df=pd.DataFrame([],columns=['pid0','pid1','pid2','process'])
        particles=[['15','11','18','tau^- -> e^- + nu_tau']]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]
        dbr=hnl.get_3body_dbr_tau(pid0,pid1,pid2)
        model.add_production_3bodydecay(
            label= "5_di",
            pid0 = pid0,
            pid1 = pid1,
            pid2 = pid2,
            br = dbr,
            generator ="pythia8",
            energy = energy,
            nsample = nsample,
            scaling = 0, 
        )
        if show_br==True:
            show_br_curve(dbr,pid0,pid1,pid2,integration="dEN",nsample=nsample)
        
    if n==3 and type_decay=="3 body pseudo":
        df=pd.DataFrame([],columns=['process','pid0','pid1','tauH (sec)','VHHp'])
        particles=[['D^+ -> \bar{K}^0 + l + N','411','-311',1040*10**(-15),0.987],
        ['D^- -> \bar{K}^0 + l + N','-411','-311',1040*10**(-15),0.987],
        ['D^0 -> K^- + l + N','421','-321',410.1*10**(-15),0.997],
        ['B^+ -> \bar{D}^0 + l + N','521','421',1.638*10**(-12),41*10**(-3)],
        ['B^0 -> D^- + l + N','511','-411',1.519*10**-12,42.2*10**-3],
        ['B^0_s -> D^-_s + l + N','531','-431',1.515*10**-12,42.2*10**-3],
        ['B^+_c -> B^0 + l + N','541','511',0.510*10**-12,0.221],
        ['B^+_c -> B^0_s + l + N','541','531',0.510*10**-12,0.987]]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]
        dbr=hnl.get_3body_dbr_pseudoscalar(pid0,pid1,pid2)
        model.add_production_3bodydecay(
        label= "5_di",
        pid0 = pid0,
        pid1 = pid1,
        pid2 = pid2,
        br = dbr,
        generator = "Pythia8",
        energy = energy,
        nsample = nsample,
        scaling = 0)
        if show_br==True:
            show_br_curve(dbr,pid0,pid1,pid2,integration="dq2dEN",nsample=nsample)

    if n==3 and type_decay=="3 body vector":
        df=pd.DataFrame([],columns=['process','pid0','pid1','tauH (sec)','VHHp'])
        particles=[['D^0 -> K*^- + e^+ + N','421','-323',410.1*10**(-15),0.967],['B^+ -> \bar{D}*^0 + e^+ + N','521','-423',1.638*10**-12,41*10**-3],['B^0 -> D*^- + e^+ + N','511','-413',1.519*10**-12,41*10**-3],['B^0_s -> D^*_s^- + e^+ + N','531','-433',1.515*10**-12,41*10**-3],['B^+_c -> B*^0 + e^+ + N','541','513',.510*10**-12,.221],['B^+_c -> B^*_s^0+ e^+ + N','541','533',.510*10**-12,41*10**-3]]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]
        dbr=hnl.get_3body_dbr_vector(pid0,pid1,pid2)
        model.add_production_3bodydecay(
        label= "5_di",
        pid0 = pid0,
        pid1 = pid1,
        pid2 = pid2,
        br = dbr,
        generator = "Pythia8",
        energy = energy,
        nsample = nsample,
        scaling = 0)
        if show_br==True:
            show_br_curve(dbr,pid0,pid1,pid2,integration="dq2dEN",nsample=nsample)


#for testing functions
#for 2 body decays
df=pd.DataFrame([],columns=['particle','pid0','tauH (sec)','MH (GeV)','VH (unitless)','fH (GeV)','process'])
particles=[['D^+','411',1040*10**(-15),1.8696,.221,.2226,'D^+ -> e^+ + N'],['D^+_s','431',500*10**(-15),1.96847,0.987,.2801, 'D_s^+ -> e^+ + N'],['B^+','521',1.638*10**(-12),5.27934,3.82*10**(-3),0.187,'B^+ -> e^+ + N'],['B^+_c','541',0.510*10**(-12),6.2749,41*10**(-3),.434, 'B^+_c -> e^+ + N']]
for n in range(len(particles)):
    df.loc[len(df)]=particles[n]

#for 2 body decays tau
df_t=pd.DataFrame([],columns=['pid0','pid1','process'])
particles=[['15','-211','tau^- -> pi^- + N'],['15','-321','tau^- -> K^- + N'],['15','-213','tau^- -> rho^- + N']]
for n in range(len(particles)):
    df_t.loc[len(df_t)]=particles[n]

#for 3 body decays tau
df_3t=pd.DataFrame([],columns=['pid0','pid1','pid2','process'])
particles=[['15','11','18','tau^- -> e^- + nu_tau + N'],['15','11','-12','tau^- -> e^- + \bar{nu}_e + N']]
for n in range(len(particles)):
    df_3t.loc[len(df_3t)]=particles[n]

#for 3 body pseudo
df_D=pd.DataFrame([],columns=['particle','pid0','pid1','tauH (sec)','VHHp'])
particles=[['D^+ -> \bar{K}^0 + l + N','411','-311',1040*10**(-15),0.987],
['D^- -> \bar{K}^0 + l + N','-411','-311',1040*10**(-15),0.987],
['D^0 -> K^- + l + N','421','-321',410.1*10**(-15),0.997],
['B^+ -> \bar{D}^0 + l + N','521','421',1.638*10**(-12),41*10**(-3)],
['B^0 -> D^- + l + N','511','-411',1.519*10**-12,42.2*10**-3],
['B^0_s -> D^-_s + l + N','531','-431',1.515*10**-12,42.2*10**-3],
['B^+_c -> B^0 + l + N','541','511',0.510*10**-12,0.221],
['B^+_c -> B^0_s + l + N','541','531',0.510*10**-12,0.987]]
for n in range(len(particles)):
    df_D.loc[len(df_D)]=particles[n]

#for 3 body vector
df=pd.DataFrame([],columns=['particle','pid0','pid1','tauH (sec)','VHHp'])
particles=[['D^0 -> K*^- + e^+ + N','421','-323',410.1*10**(-15),0.967],
['B^+ -> \bar{D}*^0 + e^+ + N','521','-423',1.638*10**-12,41*10**-3],
['B^0 -> D*^- + e^+ + N','511','-413',1.519*10**-12,41*10**-3],
['B^0_s -> D^*_s^- + e^+ + N','531','-433',1.515*10**-12,41*10**-3],
['B^+_c -> B*^0 + e^+ + N','541','513',.510*10**-12,.221],
['B^+_c -> B^*_s^0+ e^+ + N','541','533',.510*10**-12,41*10**-3]]
for n in range(len(particles)):
    df.loc[len(df)]=particles[n]

pid0='421'
pid1='-323'
pid2='-11'

add_nbodydecay(pid0,pid1,pid2,energy='14',nsample=100,generator="Pythia8",n=3,type_decay="3 body vector",show_br=True)

In [ ]:
#HNL decay attempts
#HNL decays 2 body decays
#pid1 is the meson pid2 is the smaller particle either lepton or neutrino
#from HNL_decays_1
#fH is called the decay constant
self=model
SecToGev=1./(6.582122*pow(10.,-25.))
pid1="111"
tauN=10**-5*SecToGev
GF=1.166378*10**(-5) #GeV^(-2)
coupling=1
mass=1
pid1="111"
pid2="12" #electron neutrino
br_pi_nu=f"tauN*(coupling**2*GF**2*hnl.fH(pid1)**2*mass**3/(32*np.pi))*(1-self.masses(pid1)**2/mass**2)**2"
print(eval(br_pi_nu))
pid1="411"
pid2="11"
br_H_l=f"(tauN*coupling**2*GF**2*hnl.VH(pid1)**2*hnl.fH(pid1)**2*mass**3/(32*np.pi))*((1-self.masses(pid2)**2/mass**2)**2-(self.masses(pid1)**2/mass**2)*(1+self.masses(pid2)**2/mass**2))*np.sqrt((1-((self.masses(pid1)-self.masses(pid2))**2/mass**2))*(1-((self.masses(pid1)+self.masses(pid2))**2/mass**2)))"
print(eval(br_H_l))
pid1="221"
pid2="12"
br_eta_nu=f"(tauN*coupling**2*GF**2*hnl.fH(pid1)**2*mass**3/(32*np.pi))*(1-self.masses(pid1)**2/mass**2)**2"
print(eval(br_eta_nu))
pid1="331"
pid2="12"
br_etap_nu=f"(tauN*coupling**2*GF**2*hnl.fH(pid1)**2*mass**3/(32*np.pi))*(1-self.masses(pid1)**2/mass**2)**2"
print(eval(br_etap_nu))
pid1="213"
pid2="11"
grho=0.102 #check this?
br_rho_l=f"(tauN*coupling**2*grho**2*GF**2*hnl.VH(pid1)**2*mass**3/(8*np.pi))*((1-self.masses(pid2)**2/mass**2)**2+(self.masses(pid1)**2/mass**2)*(1+((self.masses(pid2)**2-2*self.masses(pid1)**2)/mass**2))*np.sqrt((1-((self.masses(pid1)-self.masses(pid2))**2/mass**2))*(1-((self.masses(pid1)+self.masses(pid2))**2/mass**2))))"
print(eval(br_rho_l))
pid1="113"
pid2="12"
br_rho0_nu=f"(tauN*coupling**2*grho**2*GF**2*mass**3/(16*np.pi*self.masses(pid1)**2))*(1+2*self.masses(pid1)**2/mass**2)*(1-self.masses(pid1)**2/mass**2)**2"
print(eval(br_rho0_nu))

In [ ]:
#HNL decays attempt 2
#HNL decays redo.
#refer to the file All_HNL_decays?
from scipy.integrate import quad
leptons=['11','13','15']
GF=1.166378*10**(-5)
pid1="411"
br_l_P=f"(GF**2/(16*np.pi))*hnl.fH(pid1)**2*hnl.VH(pid1)**2*coupling**2*mass**3"     #N->l^- P^+, P pseudoscalar
br_nul_P=f"(GF**2/(64*np.pi))*hnl.fH(pid1)**2*coupling**2*mass**3"           #N->nu_l P^0
br_l_V=f"(GF**2/(16*np.pi))*hnl.fH(pid1)**2*hnl.VH(pid1)**2*coupling**2*mass**3"        #N->l^- V^+, V is a vector
br_nul_V=f"(GF**2/(2*np.pi))*hnl.kV(pid1)**2*hnl.fH(pid1)**2*coupling**2*mass**3"       #N->nu_l V^0
br_l1_l2_nul2=f"(GF**2/(192*np.pi**3))*coupling**2*mass**5"         #N->l1^- l_2^+ nul2
##############
#xl2=massl2/mN; l1=e,mu,tau=1,2,3; l2=e,mu,tau=1,2,3
def bra_nul1_l2_l2(mass,l1,l2):
    self=model
    xl2=self.masses('11')/mass
    x=0
    xw=0.231
    glL=-1/2+xw
    glR=xw
    GF=1.166378*10**(-5)
    lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
    I1_integrand=lambda s,xl2,x: (12/s)*(s-0*x**2-xl2**2)*(1+xl2**2-s)*np.sqrt(lamda(s,0*x**2,xl2**2))*np.sqrt(lamda(1,s,xl2**2))
    I1=quad(I1_integrand,(x*0+xl2)**2,(1-xl2)**2,args=(xl2,0))[0]
    I2_integrand=lambda s,xl2,x: (24*xl2*xl2/s)*(1-0*x**2-s)*np.sqrt(lamda(s,xl2**2,xl2**2))*np.sqrt(lamda(1,s,0*x**2))
    I2=quad(I2_integrand,(xl2+xl2)**2,(1-x*0)**2,args=(xl2,0))[0]
    delta=lambda l1,l2: 1 if l1==l2 else 0
    br_nul1_l2_l2=f"({GF}**2/(96*np.pi**3))*coupling**2*mass**5*(({glL}*{glR}+{delta(l1,l2)}*{glR})*{I2}+({glL}**2+{glR}**2+{delta(l1,l2)}*(1+2*{glL}))*{I1})"
    return(br_nul1_l2_l2)
    #return(I1,I2,delta(l1,l2),f"{delta(l1,l2)}")

##############
br_nul1_nu_nu=f"(GF**2/(96*np.pi**3))*coupling**2*mass**5"          #\sum_{l_2 = e}^{\tau} \Gamma(N->nu_l1 nu_l2 \bar{nu}_l2)


tau_N=f"6.58*10**-14*coupling**-2"
pseudoscalars_symbol=['K^0','K+', 'barK0','pi0', 'pi+', 'eta', "eta'"]
pseudoscalars=["311","-311","321","-321","111","211","-211","221","331"]            #[K^0,K^{\pm}, \bar{K}^0,\pi^0, \pi^{\pm}, \eta, \eta"] #what about D_s^+, D^+, etc.?
pseudoscalars=["311","321","111","211","221","331"] 
pseudoscalar_charged_symbols=["D^+","D^-" ,"B^+" ,"B^-" ,"B^+_c","B_c^-"]
pseudoscalars_charged=       ["411","-411","521" ,"-521","541"  ,"-541" ]
pseudoscalar_neutral_symbols=["D^0","B^0","B_s^0"]
pseudoscalars_neutral=       ["421","511","531"  ] #not sure what to do with  neutral mesons, since they dont have a ckm matrix elements, same with the light mesons
vectors_symbol=['K*0', 'barK*0', 'K*+', 'rho0', 'rho+', 'omega','phi']
vector_charged_symbols=["K^+","K^-" ]
vectors_charged=       ["321","-321"] #I believe these are the 'heavy vector mesons, not including rho0 omega phi, etc.; I took out "213","-213" (rho^+) since it doesnt have an fH it has a grho.
vectors_neutral_symbols=["K^{*0}","\bar{K}^{*0}","D^{*0}","\bar{D}^{*0}"]
vectors_neutral=        ["313"   ,"-313"        ,"423"   ,"-423"] #maybe add 443 J/psi, I just dont have fH for that one
vectors_idk=["113","223","333","443"]
#need to add br for every possible combination of l, p, nul,p etc.
#lets get it up and running for just 1
coupling=1
#the combination of coupling=1 and the extra factor*6.58*10**-14 accounts for tau_N
#br does not depend on the coupling


#for the decays N->l^{\mp} P^{\pm}
for n in range(len(pseudoscalars_charged)):
    pid1=pseudoscalars_charged[n]
    write_m_br_file(br_l_P+"*6.58*10**-14",filename=f"model/br/br_l_{pid1}.txt")

print("charged pseudoscalars done...")

#for the decays N->nu_l P^0
for n in range(len(pseudoscalars_neutral)):
    pid1=pseudoscalars_neutral[n]
    write_m_br_file(br_nul_P+"*6.58*10**-14",filename=f"model/br/br_nul_{pid1}.txt")

print("neutral pseudoscalars done")

##############
#for the decays N->l^{\mp} V^{\pm}
for n in range(len(vectors_charged)):
    pid1=vectors_charged[n]
    print(hnl.VH(pid1))
    print(hnl.fH(pid1))
    write_m_br_file(br_l_V+"*6.58*10**-14",filename=f"model/br/br_l_{pid1}.txt")

print("charged vectors done...")

#for the decays N->nu_l V^0
for n in range(len(vectors_neutral)):
    pid1=vectors_neutral[n]
    #print(hnl.VH(pid1))
    print(hnl.fH(pid1))
    write_m_br_file(br_nul_V+"*6.58*10**-14",filename=f"model/br/br_nul_{pid1}.txt")

print("neutral vectors done...")

#for the decays N->l_1^{\mp} l_2^{\pm} nu_l2
write_m_br_file(br_l1_l2_nul2+"*6.58*10**-14",filename=f"model/br/br_l1_l2_nul2.txt")

print("leptons neutrinos done 1...")

#for the decays N->nu_l1 l_2^{\mp} l_2^{\pm}
xl2=1
l1=1
l2=2
#bra_nul1_l2_l2(xl2,coupling,mass,l1,l2)
write_m_brm_file(filename=f"model/br/br_nu1_l2_l2.txt")


print("leptons neutrinos done 2...")

#for the decays N->sum(\Gamma(N->nu_l1 nu_l2 \bar{nu}_l2))
write_m_br_file(br_nul1_nu_nu+"*6.58*10**-14",filename=f"model/br/br_nul1_nu_nu.txt")

print("neutrinos done...")

write_m_ctau_file(filename="model/ctau.txt")

print("ctau file created...")

In [ ]:
#all decays, works perfectly but it turns out the decays can be simplified, lets keep this in case we need it later though
#list of mesons which the HNL can decay to.
self=model
coupling=1
GF=1.166378*10**(-5)
xw=0.231
pseudoscalars_symbols=["K^0","\bar{K}^0","K^+","K^-" ,"\pi^0","\pi^+","\pi^-","\eta","\eta'","D_s^+","D_s^-","D0" ,"D^+","D^-" ,"B^0", "B^+","B^-" ]
pseudoscalars=        ["311","-311"     ,"321","-321","111"  ,"211"  ,"-211"  ,"221","331"  ,"431"  ,"-431" ,"421","411","-411","511","521" ,"-521"]
pseudoscalars_charged_symbols=["K^+","K^-" ,"\pi^+","\pi^-","D_s^+","D_s^-","D^+","D^-" ,"B^+" ,"B^-"  ]
pseudoscalars_charged=        ["321","-321","211"  ,"-211" ,"431"  ,"-431" ,"411","-411","521" ,"-521" ]
vectors_symbols=["K^{*0}","\bar{K}^{*0}","K^{*+}","K^{*-}","\rho_0","\rho^+","\rho^-","\omega","\phi","D^*0" ]
vectors=        ["313"   ,"-313"        ,"323"   ,"-323"  ,"113"   ,"213"   ,"-213"  ,"223"   ,"333" ,"423"  ]
vectors_charged_symbols=["K^{*+}","K^{*-}"]
vectors_charged=        ["323"   ,"-323"  ] #took out omega and phi, not sure where to find data, their lifetimes are super small 10^-23 roughly and they decay into channels we have implemented
#HNL decays (new) 2 body  
#for N->pi^0 nu_alpha
leptons=["11","13","15"]
nu_alpha=["12","14","16"]
pid1="111"
br_pi0_nu=f"(coupling**2/(32*np.pi))*GF**2*hnl.fH(pid1)**2*mass**3*(1-self.masses(pid1)**2/mass**2)**2"
for n in range(len(nu_alpha)):
    pid2=nu_alpha[n]
    write_m_br_file(br_pi0_nu,filename=f"model/br/width_2body_pi0_{pid2}.txt")
print('N->pi^0 nu_alpha done...')

#for N->H+ l_alpha-
#I assume H+ is a pseudoscalar, this may be wrong, lets crosscheck this formula with other sources later
'''br_H_l=f"(coupling**2/(16*np.pi))*GF**2*hnl.VH(pid1)**2*hnl.fH(pid1)**2*mass**3*((1-self.masses(pid2)**2/mass**2)**2-(self.masses(pid1)**2/mass**2)*(1+self.masses(pid2)**2/mass**2))*np.sqrt((1-(self.masses(pid1)-self.masses(pid2))**2/mass**2)*(1-(self.masses(pid1)+self.masses(pid2))**2/mass**2))"
for n in range(len(pseudoscalars_charged)):
    pid1=pseudoscalars_charged[n]
    for m in range(len(leptons)):
        pid2=leptons[m]
        if int(pid1)<0:
            pid2="-" + pid2
        write_m_br_file(br_H_l,filename=f"model/br/width_{pid1}_{pid2}.txt")'''

#here I am going to assume H+ includes both pseudoscalars and vectors, since the total width is not nearly large enough yet
br_H_l=f"(coupling**2/(16*np.pi))*GF**2*hnl.VH(pid1)**2*hnl.fH(pid1)**2*mass**3*((1-self.masses(pid2)**2/mass**2)**2-(self.masses(pid1)**2/mass**2)*(1+self.masses(pid2)**2/mass**2))*np.sqrt((1-(self.masses(pid1)-self.masses(pid2))**2/mass**2)*(1-(self.masses(pid1)+self.masses(pid2))**2/mass**2))"
hadrons_charged=pseudoscalars_charged+vectors_charged
for n in range(len(hadrons_charged)):
    pid1=hadrons_charged[n]
    print(pid1)
    for m in range(len(leptons)):
        pid2=leptons[m]
        if int(pid1)<0:
            pid2="-" + pid2
        write_m_br_file(br_H_l,filename=f"model/br/width_2body_{pid1}_{pid2}.txt")
print('N->H+ l_alpha- done...')

#for N->eta nu_alpha
pid1="221"
br_eta_nu=f"(coupling**2/(32*np.pi))*GF**2*hnl.fH(pid1)**2*mass**3*(1-self.masses(pid1)**2/mass**2)**2"
for n in range(len(nu_alpha)):
    pid2=nu_alpha[n]
    write_m_br_file(br_eta_nu,filename=f"model/br/width_2body_eta_{pid2}.txt")
print('N->eta nu_alpha done...')

#for N->eta' nu_alpha
pid1="331"
br_etap_nu=f"(coupling**2/(32*np.pi))*GF**2*hnl.fH(pid1)**2*mass**3*(1-self.masses(pid1)**2/mass**2)**2"
for n in range(len(nu_alpha)):
    pid2=nu_alpha[n]
    write_m_br_file(br_etap_nu,filename=f"model/br/width_2body_etap_{pid2}.txt")
print("N->eta' nu_alpha done...")

#for N->rho+ l_alpha-
grho=0.102
pid1="213"
br_rho_l=f"(coupling**2/(8*np.pi))*(grho**2/self.masses(pid1)**2)*GF**2*hnl.VH(pid1)**2*mass**3*((1-self.masses(pid2)**2/mass**2)**2+(self.masses(pid1)**2/mass**2)*(1+(self.masses(pid2)**2-2*self.masses(pid1)**2)/mass**2))*np.sqrt((1-(self.masses(pid1)-self.masses(pid2))**2/mass**2)*(1-(self.masses(pid1)+self.masses(pid2))**2/mass**2))"
for n in range(len(leptons)):
    pid2=leptons[n]
    write_m_br_file(br_rho_l,filename=f"model/br/width_2body_rho_{pid2}.txt")
pid1="-213"
for n in range(len(leptons)):
    pid2=leptons[n]
    write_m_br_file(br_rho_l,filename=f"model/br/width_2body_rho_{pid2}.txt")
print('N->rho+ l_alpha- done...')

#for N->rho0 nu_alpha
pid1="113"
pid2="11"
br_rho0_nu=f"(coupling**2/(16*np.pi))*(grho**2/self.masses(pid1)**2)*GF**2*mass**3*(1+2*self.masses(pid1)**2/mass**2)*(1-self.masses(pid1)**2/mass**2)**2"
for n in range(len(nu_alpha)):
    pid2=nu_alpha[n]
    write_m_br_file(br_rho0_nu,filename=f"model/br/width_2body_rho0_{pid2}.txt")
print('N->rho0 nu_alpha done...')

#HNL decays 3 body
#for N->sum_{alpha beta} nu_alpha bar{nu}_beta nu_beta
#pid_e="11"
#pid_mu="13"
#pid_tau="15"
#br_nu_bnu_nu=f"(GF**2*mass**5/(192*np.pi**3))*(hnl.vcoupling[pid_e]**2+hnl.vcoupling[pid_mu]**2+hnl.vcoupling[pid_tau]**2)"

#modifying previous decay to not sum over all channels
#for N->sum_{alpha beta} nu_alpha bar{nu}_beta nu_beta
br_nu_bnu_nu=f"(GF**2*mass**5/(192*np.pi**3))*(coupling**2)"
for n in range(len(nu_alpha)):
    for m in range(len(nu_alpha)):
        pid1=nu_alpha[n]
        pid2=nu_alpha[m]
        write_m_br_file(br_nu_bnu_nu,filename=f"model/br/width_3body_nua_bnub_nub_{pid1}_{pid2}.txt")
print('3 body N->sum_{alpha beta} nu_alpha bar{nu}_beta nu_beta done...')


#for N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta
pid1="11"
pid2="-13"
pid3="14"
xl1=np.max([self.masses(pid1),self.masses(pid2)])
br_la_lb_nub=f"(GF**2*mass**5/(192*np.pi**3))*coupling**2*(1-8*xl1**2+8*xl1**6-xl1**8-12*xl1**4*np.log(xl1**2))"
for a in range(len(leptons)):
    for b in range(len(leptons)):
        if a!=b:
            pid1=leptons[a]
            pid2=leptons[b]
            pid2="-"+pid2
            pid3=nu_alpha[b]
            write_m_br_file(br_la_lb_nub,filename=f"model/br/width_3body_la_lb_nub_{pid1}_{pid2}_{pid3}.txt")
print('3 body N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta done...')




#for N->l_beta^+ l_beta^- nu_alpha 
#pid1="-13"
#pid2="13"
#pid3="16"
def analyze_pid1_pid3(pid1,pid3):
    if pid1 in ["11","-11"]:
        beta=1
    if pid1 in ["13","-13"]:
        beta=2
    if pid1 in ["15","-15"]:
        beta=3
    if pid3 in ["12","-12"]:
        alpha=1
    if pid3 in ["14","-14"]:
        alpha=2
    if pid3 in ["16","-16"]:
        alpha=3
    return(alpha,beta)
def calc_br_lb_lb_nua(pid1,pid2,pid3):
    delta=lambda l1,l2: 1 if l1==l2 else 0
    xl=f"self.masses(pid1)/mass"
    xw=f"0.231" #xw=sin(theta_w)^2
    L=f"np.log((1-3*{xl}**2-(1-{xl}**2)*np.sqrt(1-4*{xl}**2))/({xl}**2*(1+np.sqrt(1-4*{xl}**2))))"
    C1=f"(1/4)*(1-4*xw+8*xw**2)"
    C2=f"(1/2)*xw*(2*xw-1)"
    C3=f"(1/4)*(1+4*xw+8*xw**2)"
    C4=f"(1/2)*xw*(2*xw+1)"
    alpha,beta=analyze_pid1_pid3(pid1,pid3)
    br_lb_lb_nua=f"(GF**2*mass**5/(192*np.pi**3))*coupling**2*(({C1}*(1-{delta(alpha,beta)})+{C3}*{delta(alpha,beta)})*((1-14*{xl}**2-2*{xl}**4-12*{xl}**6)*np.sqrt(1-4*{xl}**2)+12*{xl}**4*({xl}**4-1)*{L})+4*({C2}*(1-{delta(alpha,beta)})+{C4}*{delta(alpha,beta)})*({xl}**2*(2+10*{xl}**2-12*{xl}**4)*np.sqrt(1-4*{xl}**2)+6*{xl}**4*(1-2*{xl}**2+2*{xl}**4)*{L}))"
    return(br_lb_lb_nua)
for b in range(len(leptons)):
    for a in range(len(nu_alpha)):
        pid1=leptons[b]
        pid1="-"+pid1
        pid2=leptons[b]
        pid3=nu_alpha[a]
        br_lb_lb_nua=calc_br_lb_lb_nua(pid1,pid2,pid3)
        write_m_br_file(br_lb_lb_nua,filename=f"model/br/width_3body_lb_lb_nua_{pid1}_{pid2}_{pid3}.txt")
print('3 body N->l_beta^+ l_beta^- nu_alpha done...')




#HNL decays 3 body (quarks)
#for N->U bar{D} l_alpha^-
#pid1=U; pid2=bar{D}; pid3=l_alpha^-
from scipy.integrate import quad
pid1="2"
pid2="1"
pid3="11"
quarks_u_symbols=["u","c","t"]
quarks_u=        ["2","4"]
quarks_d_symbols=["d","s","b"]      #skipped top quark since they are very heavy
quarks_d=        ["1","3","5"]

I_integrand=lambda x,xl,xd,xu: (12/x)*(x-xl**2-xd**2)*(1+xu**2-x)*np.sqrt(lamda(x,xl**2,xd**2))*np.sqrt(lamda(1,x,xu**2))
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
def calc_br_u_bd_l(pid1,pid2,pid3):
    xd=f"self.masses(pid2)/mass"
    xu=f"self.masses(pid1)/mass"
    xl=f"self.masses(pid3)/mass"
    I=f"quad(I_integrand,({xd}+{xl})**2,(1-{xu})**2,args=({xl},{xd},{xu}))[0]"
    br_u_d_l=f"(3*hnl.VHHp(pid1,pid2)**2*GF**2*mass**5*coupling**2/(192*np.pi**3))*{I}"
    #return(br_u_d_l)
    return(br_u_d_l)

for u in range(len(quarks_u)):
    for d in range(len(quarks_d)):
        for a in range(len(leptons)):
            pid1=quarks_u[u]
            pid2=quarks_d[d]
            pid3=leptons[a]
            br_u_d_l=calc_br_u_bd_l(pid1,pid2,pid3)
            write_m_br_file(br_u_d_l,filename=f"model/br/width_3body_u_d_l_{pid1}_{pid2}_{pid3}.txt")
print('3 body N->U bar{D} l_alpha^- done...')

#HNL decays 3 body (quarks)
#for N-> q \bar{q} nu_l
nu_alpha=["12","14","16"]
GF=1.166378*10**(-5)
pid1="2"
mass=1
coupling=1
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
x=0
xq=f"self.masses(pid1)/mass"
y=xq
z=xq
xw=f"0.231" #xw=sin(theta_w)^2
I1_integrand=lambda s,x,y,z: (12/s)*(s-x**2-y**2)*(1+z**2-s)*np.sqrt(lamda(s,x**2,y**2))*np.sqrt(lamda(1,s,z**2))
I2_integrand=lambda s,x,y,z: (24*y*z/s)*(1+x**2-s)*np.sqrt(lamda(s,y**2,z**2))*np.sqrt(lamda(1,s,x**2))

def calc_br_q_bq_nu(pid1):
    if pid1 in quarks_u:
        gL=f"(1/2-(2/3)*{xw})"
        gR=f"(-(2/3)*{xw})" #article had 2 g_R^U so I assumed one was supposed to have a D
    if pid1 in quarks_d:
        gL=f"(-1/2+(1/3)*{xw})"
        gR=f"((1/3)*{xw})"
    I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
    I2=f"quad(I2_integrand,({y}+{z})**2,(1-{x})**2,args=({x},{y},{z}))[0]"
    br_q_bq_nu=f"(coupling**2*GF**2*mass**5/(32*np.pi**3))*({gL}*{gR}*{I2}+({gL}**2+{gR}**2)*{I1})"
    return(br_q_bq_nu)

for q in range(len(quarks_u+quarks_d)):
    pid1=(quarks_u+quarks_d)[q]
    pid2="-"+pid1
    for a in range(len(nu_alpha)):
        pid3=nu_alpha[a]
        br_q_bq_nu=calc_br_q_bq_nu(pid1)
        write_m_br_file(br_q_bq_nu,filename=f"model/br/width_3body_u_bu_l_{pid1}_{pid2}_{pid3}.txt")
print('3 body N-> q \bar{q} nu_alpha done...')



In [ ]:
#decay modes not in use
#exclusive hadronic channels
#took out because pi0 is a neutral pseudo
'''
#for N->pi^0 nu_alpha
leptons=["11","13","15"]
nu_alpha=["12","14","16"]
pid1="111"
mass_produced=self.masses(pid1)
#no factor of 2 added (might need it)
br_pi0_nu="2*"+f"(coupling**2/(32*np.pi))*GF**2*hnl.fH(pid1)**2*mass**3*(1-self.masses(pid1)**2/mass**2)**2"
for n in range(len(nu_alpha)):
    pid2=nu_alpha[n]
    write_m_br_file(br_pi0_nu,mass_produced,filename=f"model/width/width_2body_ex_pi0_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print('N->pi^0 nu_alpha done...')'''

#took out because eta is a neutral pseudo
'''#for N->eta nu_alpha
pid_start="111"
pid_end="331"
start_mass=self.masses(pid_start)
end_mass=self.masses(pid_end)
pid1="221"
#no factor of 2 added (probably need it)
br_eta_nu="2*"+f"(coupling**2/(32*np.pi))*GF**2*hnl.fH(pid1)**2*mass**3*(1-self.masses(pid1)**2/mass**2)**2"
mass_produced=self.masses(pid1)
for n in range(len(nu_alpha)):
    pid2=nu_alpha[n]
    write_m_br_file(br_eta_nu,mass_produced,filename=f"model/width/width_2body_ex_eta_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print('N->eta nu_alpha done...')'''

#felix doesnt include this channel
'''
#for N->eta' nu_alpha
pid1="331"
#no factor of 2 added (might need to add it)
br_etap_nu=f"(coupling**2/(32*np.pi))*GF**2*hnl.fH(pid1)**2*mass**3*(1-self.masses(pid1)**2/mass**2)**2"
mass_produced=self.masses(pid1)
for n in range(len(nu_alpha)):
    pid2=nu_alpha[n]
    write_m_br_file(br_etap_nu,mass_produced,filename=f"model/width/width_2body_ex_etap_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print("N->eta' nu_alpha done...")'''

#taking this one out because rho+ is a vector
'''
#for N->rho+ l_alpha-
grho=0.102
pid1="213"
br_rho_l="2*"+f"(coupling**2/(8*np.pi))*(grho**2/self.masses(pid1)**2)*GF**2*hnl.VH(pid1)**2*mass**3*((1-self.masses(pid2)**2/mass**2)**2+(self.masses(pid1)**2/mass**2)*(1+(self.masses(pid2)**2-2*self.masses(pid1)**2)/mass**2))*np.sqrt((1-(self.masses(pid1)-self.masses(pid2))**2/mass**2)*(1-(self.masses(pid1)+self.masses(pid2))**2/mass**2))"
for n in range(len(leptons)):
    pid2=leptons[n]
    mass_produced=self.masses(pid1)+self.masses(pid2)
    write_m_br_file(br_rho_l,mass_produced,filename=f"model/width/width_2body_ex_rho_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)



#for N->rho0 nu_alpha
pid1="113"
pid2="11"
#no factor of 2 added (should probably add it)
br_rho0_nu="2*"+f"(coupling**2/(16*np.pi))*(grho**2/self.masses(pid1)**2)*GF**2*mass**3*(1+2*self.masses(pid1)**2/mass**2)*(1-self.masses(pid1)**2/mass**2)**2"
mass_produced=self.masses(pid1)
for n in range(len(nu_alpha)):
    pid2=nu_alpha[n]
    write_m_br_file(br_rho0_nu,mass_produced,filename=f"model/width/width_2body_ex_rho0_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print('N->rho0 nu_alpha done...')'''

In [ ]:
#useful functions to create and read files, I think all of these might be redundant
#this portion will take a branching fraction and write to a file in the form m br
#takes a number and chops off part of it according to the precision, or delta
def chop(expr, delta=10**-2):
    return np.ma.masked_inside(expr, -delta, delta).filled(0)
#for the decays of HNL, takes a branching ratio and creates a file of mass br
def write_m_br_file(br,mass_produced,filename="model/pi_nu.txt"):
    f = open(filename, "w+")
    f.close()
    f = open(filename, "a")
    #mass=1
    mass=.01
    delm=.01
    steps=int(10/.01)-1

    for n in range(steps):
        #f.write(f"{m}  {br}\n")
        #print(eval(br))
        if mass>mass_produced:
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(br)]))
        else:
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,0]))

        mass=chop(mass+delm)
    f.close()
    print('file created...')
    #a=.0134155

#same as write_m_br_file except for mass between, start_mass, end_mass is for the starting mass for br_had and ending mass for br_had
def write_m_br_had_file(br,br_had,mass_produced,start_mass,end_mass,filename="model/pi_nu.txt"):
    f = open(filename, "w+")
    f.close()
    f = open(filename, "a")
    #mass=1




    mass=start_mass
    delm=.01
    steps=int((end_mass-start_mass)/.01)+1
    for n in range(steps):
        #f.write(f"{m}  {br}\n")
        #print(eval(br))
        if mass>mass_produced:
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(br_had)]))
        else:
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,0]))

        mass=chop(mass+delm)






    mass=end_mass
    delm=.01
    steps=int((10-end_mass)/.01)+1
    for n in range(steps):
        #f.write(f"{m}  {br}\n")
        #print(eval(br))
        if mass>mass_produced:
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(br)]))
        else:
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,0]))

        mass=chop(mass+delm)
    f.close()
    print('file created...')
    #a=.0134155

#this is for the decay N->nu_l1 l2 l2
def write_m_brm_file(filename="model/pi_nu.txt"):
    f = open(filename, "w+")
    f.close()
    f = open(filename, "a")
    #mass=1
    mass=.01
    br=bra_nul1_l2_l2(mass,l1,l2)+"*6.58*10**-14"
    delm=.01
    steps=int(10/.01)-1
    for n in range(steps):
        #f.write(f"{m}  {br}\n")
        #print(eval(br))
        f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(br)]))
        mass=chop(mass+delm)
    f.close()
    print('file created...')

def write_m_ctau_file(filename="model/ctau.txt"):
    f = open(filename, "w+")
    f.close()
    f = open(filename, "a")
    #mass=1
    mass=.01
    coupling=1
    tau_N=f"6.58*10**-14*coupling**-2"
    ctau=tau_N+"*3*10**8"
    delm=.01
    steps=int(10/.01)-1
    for n in range(steps):
        #f.write(f"{m}  {br}\n")
        #print(eval(br))
        f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(ctau)]))
        mass=chop(mass+delm)
    f.close()
    print('file created...')

#write_m_br_file(br)

In [ ]:
import pandas as pd
def add_br_files(files):
    counter=0
    for n in range(len(br_files)):
        for m in range(len(files)):
            if br_files[n][0:len(files[m])] in files:
                counter+=1
                file_name=br_files[n]
                m_width_df_temp = pd.read_csv(f"model/br/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False)
                m_width_df_temp['width'] = m_width_df_temp['width'].fillna(0)
                if counter==1:
                    width=np.array(m_width_df_temp['width'].tolist())
                if counter!=1:
                    width_temp=np.array(m_width_df_temp['width'].tolist())
                    width+= width_temp
    mass=np.array(m_width_df_temp['mass'].tolist())
    return(mass,width)
#3 nu curve
filename="model/br/br_3body_lep_sum_nua_bnub_nub.txt"
m3nu,br3nu=read_file(filename,name_col1="x",name_col2="y")

#hadron curve (done)
br_files=os.listdir("model/br/")
had_file_char="br_3body_inc"
had_file_char1="br_2body_ex"
files_had=["br_3body_inc","br_2body_ex"]
mhad,brhad=add_br_files(files_had)

#eev curve (done)
files_eev=["br_3body_lep_lb_lb_nua_-11_11"]
meev,breev=add_br_files(files_eev)

#emuv curve (not done)
files_emuv=["br_3body_lep_la_lb_nub_11_-13_14"]
memuv,bremuv=add_br_files(files_emuv)

#mumuv (done)
files_mumuv=["br_3body_lep_lb_lb_nua_-13_13_12"]
mmumuv,brmumuv=add_br_files(files_mumuv)

#etv (not done)
files_etv=["br_3body_lep_la_lb_nub_11_-15_16"]
metv,bretv=add_br_files(files_etv)

#ttv (done)
files_ttv=["br_3body_lep_lb_lb_nua_-15_15_12"]
mttv,brttv=add_br_files(files_ttv)

plt.plot(m3nu,br3nu,label=r"3$\nu$")
plt.plot(mhad,brhad,label=r"hadrons")
plt.plot(meev,breev,label=r"ee$\nu$")
plt.plot(memuv,bremuv,label=r"e$\mu \nu$")
plt.plot(mmumuv,brmumuv,label=r"$\mu \mu \nu$")
plt.plot(metv,bretv,label=r"$e \tau \nu$")
plt.plot(mttv,brttv, label=r"$\tau \tau \nu$")

plt.legend()

plt.xlabel(r"$M_N$")
plt.ylabel("Br")
plt.xlim(.1,10)
plt.ylim(10**-3,2)
plt.xscale("log")
plt.yscale("log")
plt.show()

In [ ]:
#generates the ratio of the two spectra for tau that felix was looking for.
df1=pd.read_csv(f"{src_path}/files/hadrons/14TeV/Pythia8/Pythia8_14TeV_-15.txt",delimiter='\t',engine='python',names=['logtheta','logp','weight'],index_col=False)
df2=pd.read_csv(f"{src_path}/files/hadrons/14TeV/Pythia8/Pythia8_14TeV_15.txt",delimiter=' ',engine='python',names=['logtheta','logp','weight'],index_col=False)

#take the intersection in the logtheta logp plane, then we can take their ratio and set all nan and inf values to 0 by definition
df3 = pd.merge(df1, df2, how ='inner', on =['logtheta', 'logp'])
df3['weight']=df3['weight_y']/df3['weight_x']
df3['weight']=df3['weight'].fillna(0)
df3['weight'].replace([np.inf, -np.inf], 0, inplace=True)
df3=df3.drop(columns=['weight_x','weight_y'])

#print(df3['weight'].tolist())
df3.to_csv(f"{src_path}/files/hadrons/14TeV/Pythia8/Pythia8_14TeV_15.txt", header=None, index=None, sep='\t')

In [ ]:
#unused but functional HNL decay channels, repeats of above except with different formulas
#for N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta
xl1=f"(np.max([self.masses(pid1),self.masses(pid2)])/mass)"
br_la_lb_nub=f"(GF**2*mass**5/(192*np.pi**3))*hnl.vcoupling[str(abs(int(pid1)))]**2*(1-8*{xl1}**2+8*{xl1}**6-{xl1}**8-12*{xl1}**4*np.log({xl1}**2))"
for a in range(len(leptons)):
    for b in range(len(leptons)):
        if a!=b:
            pid1=leptons[a]
            pid2=leptons[b]
            pid2="-"+pid2
            pid3=nu_alpha[b]
            xl1=f"(np.max([self.masses(pid1),self.masses(pid2)])/mass)"
            mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
            write_m_br_file(br_la_lb_nub,mass_produced,filename=f"model/width/width_3body_lep_la_lb_nub_{pid1}_{pid2}_{pid3}.txt")

#for N->l_beta^+ l_beta^- nu_alpha 
import sympy as smp
import mpmath as mp
def analyze_pid1_pid3(pid1,pid3):
    if pid1 in ["11","-11"]:
        beta=1
    if pid1 in ["13","-13"]:
        beta=2
    if pid1 in ["15","-15"]:
        beta=3
    if pid3 in ["12","-12"]:
        alpha=1
    if pid3 in ["14","-14"]:
        alpha=2
    if pid3 in ["16","-16"]:
        alpha=3
    return(alpha,beta)
def calc_br_lb_lb_nua(pid1,pid2,pid3):
    delta=lambda l1,l2: 1 if l1==l2 else 0
    xl=f"(self.masses(pid1)/mass)"
    xw=f"(0.231)" #xw=sin(theta_w)^2
    L=f"np.log((1-3*{xl}**2-(1-{xl}**2)*np.sqrt(1-4*{xl}**2))/({xl}**2*(1+np.sqrt(1-4*{xl}**2))))"
    C1=f"(1/4)*(1-4*{xw}+8*{xw}**2)"
    C2=f"(1/2)*{xw}*(2*{xw}-1)"
    C3=f"(1/4)*(1+4*{xw}+8*{xw}**2)"
    C4=f"(1/2)*{xw}*(2*{xw}+1)"
    alpha,beta=analyze_pid1_pid3(pid1,pid3)
    coupling=f"hnl.vcoupling[str(abs(int(pid3))-1)]"
    br_lb_lb_nua=f"(GF**2*mass**5/(192*np.pi**3))*coupling**2*(({C1}*(1-{delta(alpha,beta)})+{C3}*{delta(alpha,beta)})*((1-14*{xl}**2-2*{xl}**4-12*{xl}**6)*np.sqrt(1-4*{xl}**2)+12*{xl}**4*({xl}**4-1)*{L})+4*({C2}*(1-{delta(alpha,beta)})+{C4}*{delta(alpha,beta)})*({xl}**2*(2+10*{xl}**2-12*{xl}**4)*np.sqrt(1-4*{xl}**2)+6*{xl}**4*(1-2*{xl}**2+2*{xl}**4)*{L}))"
    return(br_lb_lb_nua)
for b in range(len(leptons)):
    for a in range(len(nu_alpha)):
        pid1=leptons[b]
        pid1="-"+pid1
        pid2=leptons[b]
        pid3=nu_alpha[a]
        mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
        br_lb_lb_nua=calc_br_lb_lb_nua(pid1,pid2,pid3)
        write_m_br_file(br_lb_lb_nua,mass_produced,filename=f"model/width/width_3body_lep_lb_lb_nua_{pid1}_{pid2}_{pid3}.txt")
print('3 body N->l_beta^+ l_beta^- nu_alpha done...')

In [ ]:
#HNL decays 3 body (quarks)
#for N->U bar{D} l_alpha^-
#pid1=U; pid2=bar{D}; pid3=l_alpha^-
mass=1
from scipy.integrate import quad
#pid1="2"
#pid2="1"
#pid3="11"
quarks_u_symbols=["u","c","t"]
quarks_u=        ["2","4"]
quarks_d_symbols=["d","s","b"]      #skipped top quark since it is very heavy
quarks_d=        ["1","3","5"]
pid1="211"
#br_H_l=f"(coupling**2/(16*np.pi))*GF**2*hnl.VH(pid1)**2*hnl.fH(pid1)**2*mass**3*((1-self.masses(pid2)**2/mass**2)**2-(self.masses(pid1)**2/mass**2)*(1+self.masses(pid2)**2/mass**2))*np.sqrt((1-(self.masses(pid1)-self.masses(pid2))**2/mass**2)*(1-(self.masses(pid1)+self.masses(pid2))**2/mass**2))"
I_integrand=lambda s,x,y,z: (12/s)*(s-x**2-y**2)*(1+z**2-s)*np.sqrt(lamda(s,x**2,y**2))*np.sqrt(lamda(1,s,z**2))
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
def calc_br_u_bd_l(pid1,pid2,pid3):
    xd=f"(self.masses(pid2)/mass)"
    xu=f"(self.masses(pid1)/mass)"
    xl=f"(self.masses(pid3)/mass)"
    x=xl
    y=xu
    z=xd
    I=f"quad(I_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
    coupling=f"hnl.vcoupling[str(abs(int(pid3)))]"
    #this one already got multiplied by 2 to account for majorana 
    br_u_d_l="2*"+f"(hnl.VHHp(pid1,pid2)**2*GF**2*mass**5*coupling**2/(32*np.pi**3))*{I}"
    return(br_u_d_l)

for u in range(len(quarks_u)):
    for d in range(len(quarks_d)):
        for a in range(len(leptons)):
            pid1=quarks_u[u]
            pid2=quarks_d[d]
            pid3=leptons[a]
            mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
            br_u_d_l=calc_br_u_bd_l(pid1,pid2,pid3)
            write_m_br_file(br_u_d_l,mass_produced,filename=f"model/width/width_3body_inc_u_d_l_{pid1}_{pid2}_{pid3}.txt",start_mass=end_mass,end_mass=10)
print('3 body N->U bar{D} l_alpha^- done...')

#HNL decays 3 body (quarks)
#inclusive decays
#for N-> q \bar{q} nu_l
nu_alpha=["12","14","16"]
GF=1.166378*10**(-5)
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
x=0
xq=f"(self.masses(pid1)/mass)"
y=xq
z=xq
xw=f"(0.231)" #xw=sin(theta_w)^2
I1_integrand=lambda s,x,y,z: (12/s)*(s-x**2-y**2)*(1+z**2-s)*np.sqrt(lamda(s,x**2,y**2))*np.sqrt(lamda(1,s,z**2))
I2_integrand=lambda s,x,y,z: (24*y*z/s)*(1+x**2-s)*np.sqrt(lamda(s,y**2,z**2))*np.sqrt(lamda(1,s,x**2))

def calc_br_q_bq_nu(pid1,pid3):
    if pid1 in quarks_u:
        gL=f"(1/2-(2/3)*{xw})"
        gR=f"(-(2/3)*{xw})" #article had 2 g_R^U so I assumed one was supposed to have a D
    if pid1 in quarks_d:
        gL=f"(-1/2+(1/3)*{xw})"
        gR=f"((1/3)*{xw})"
    I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
    I2=f"quad(I2_integrand,({y}+{z})**2,(1-{x})**2,args=({x},{y},{z}))[0]"
    coupling=f"hnl.vcoupling[str(abs(int(pid3))-1)]"
    #no factor of 2 since the charge conjugate process is the same process (I think the source already included factor of 2 here)
    br_q_bq_nu="(1/2)*"+f"(coupling**2*GF**2*mass**5/(32*np.pi**3))*({gL}*{gR}*{I2}+({gL}**2+{gR}**2)*{I1})"
    return(br_q_bq_nu)

pid1="111"
br_pi0_nu="2*"+f"(coupling**2/(32*np.pi))*GF**2*hnl.fH(pid1)**2*mass**3*(1-self.masses(pid1)**2/mass**2)**2"

for q in range(len(quarks_u+quarks_d)):
    pid1=(quarks_u+quarks_d)[q]
    pid2="-"+pid1
    for a in range(len(nu_alpha)):
        pid3=nu_alpha[a]
        mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
        br_q_bq_nu=calc_br_q_bq_nu(pid1,pid3)
        write_m_br_file(br_q_bq_nu,mass_produced,filename=f"model/width/width_3body_inc_u_bu_l_{pid1}_{pid2}_{pid3}.txt",start_mass=end_mass,end_mass=10)
print('3 body N-> q \bar{q} nu_alpha done...')

In [ ]:
#this part analyzes how well inclusive and exclusive methods agree (old)

#HNL decays 3 body (quarks)
#for N->U bar{D} l_alpha^-
#pid1=U; pid2=bar{D}; pid3=l_alpha^-
from scipy.integrate import quad
#pid1="2"
#pid2="1"
#pid3="11"
quarks_u_symbols=["u","c","t"]
quarks_u=        ["2","4"]
quarks_d_symbols=["d","s","b"]      #skipped top quark since it is very heavy
quarks_d=        ["1","3","5"]
pid1="211"
br_H_l=f"(coupling**2/(16*np.pi))*GF**2*hnl.VH(pid1)**2*hnl.fH(pid1)**2*mass**3*((1-self.masses(pid2)**2/mass**2)**2-(self.masses(pid1)**2/mass**2)*(1+self.masses(pid2)**2/mass**2))*np.sqrt((1-(self.masses(pid1)-self.masses(pid2))**2/mass**2)*(1-(self.masses(pid1)+self.masses(pid2))**2/mass**2))"
I_integrand=lambda s,x,y,z: (12/s)*(s-x**2-y**2)*(1+z**2-s)*np.sqrt(lamda(s,x**2,y**2))*np.sqrt(lamda(1,s,z**2))
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
def calc_br_u_bd_l(pid1,pid2,pid3):
    xd=f"(self.masses(pid2)/mass)"
    xu=f"(self.masses(pid1)/mass)"
    xl=f"(self.masses(pid3)/mass)"
    x=xl
    y=xu
    z=xd
    I=f"quad(I_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
    coupling=f"hnl.vcoupling[str(abs(int(pid3)))]"
    br_u_d_l=f"(hnl.VHHp(pid1,pid2)**2*GF**2*mass**5*{coupling}**2/(32*np.pi**3))*{I}"
    return(br_u_d_l)

for u in range(len(quarks_u)):
    for d in range(len(quarks_d)):
        for a in range(len(leptons)):
            pid1=quarks_u[u]
            pid2=quarks_d[d]
            pid3=leptons[a]
            mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
            br_u_d_l=calc_br_u_bd_l(pid1,pid2,pid3)
            write_m_br_file(br_u_d_l,mass_produced,filename=f"model/width_temp/width_3body_inc_u_d_l_{pid1}_{pid2}_{pid3}.txt",start_mass=end_mass,end_mass=10)
print('3 body N->U bar{D} l_alpha^- done...')
#HNL decays 3 body (quarks)
#inclusive decays
#for N-> q \bar{q} nu_l
start_mass=.01
end_mass=10
nu_alpha=["12","14","16"]
GF=1.166378*10**(-5)
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
x=0
xq=f"(self.masses(pid1)/mass)"
y=xq
z=xq
xw=f"(0.231)" #xw=sin(theta_w)^2
I1_integrand=lambda s,x,y,z: (12/s)*(s-x**2-y**2)*(1+z**2-s)*np.sqrt(lamda(s,x**2,y**2))*np.sqrt(lamda(1,s,z**2))
I2_integrand=lambda s,x,y,z: (24*y*z/s)*(1+x**2-s)*np.sqrt(lamda(s,y**2,z**2))*np.sqrt(lamda(1,s,x**2))

def calc_br_q_bq_nu(pid1,pid3):
    if pid1 in quarks_u:
        gL=f"(1/2-(2/3)*{xw})"
        gR=f"(-(2/3)*{xw})" #article had 2 g_R^U so I assumed one was supposed to have a D
    if pid1 in quarks_d:
        gL=f"(-1/2+(1/3)*{xw})"
        gR=f"((1/3)*{xw})"
    I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
    I2=f"quad(I2_integrand,({y}+{z})**2,(1-{x})**2,args=({x},{y},{z}))[0]"
    coupling=f"hnl.vcoupling[str(abs(int(pid3))-1)]"
    br_q_bq_nu="(1/2)*"+f"({coupling}**2*GF**2*mass**5/(32*np.pi**3))*({gL}*{gR}*{I2}+({gL}**2+{gR}**2)*{I1})"
    return(br_q_bq_nu)

pid1="111"
br_pi0_nu=f"(coupling**2/(32*np.pi))*GF**2*hnl.fH(pid1)**2*mass**3*(1-self.masses(pid1)**2/mass**2)**2"

for q in range(len(quarks_u+quarks_d)):
    pid1=(quarks_u+quarks_d)[q]
    pid2="-"+pid1
    for a in range(len(nu_alpha)):
        pid3=nu_alpha[a]
        mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
        br_q_bq_nu=calc_br_q_bq_nu(pid1,pid3)
        write_m_br_file(br_q_bq_nu,mass_produced,filename=f"model/width_temp/width_3body_inc_u_bu_l_{pid1}_{pid2}_{pid3}.txt",start_mass=start_mass,end_mass=end_mass)
print('3 body N-> q \bar{q} nu_alpha done...')

#exclusive hadronic channels
#for N->pi^0 nu_alpha
leptons=["11","13","15"]
nu_alpha=["12","14","16"]
pid1="111"
mass_produced=self.masses(pid1)
coupling=1
br_pi0_nu=f"(coupling**2/(32*np.pi))*GF**2*hnl.fH(pid1)**2*mass**3*(1-self.masses(pid1)**2/mass**2)**2"
for n in range(len(nu_alpha)):
    pid2=nu_alpha[n]
    write_m_br_file(br_pi0_nu,mass_produced,filename=f"model/width_temp/width_2body_ex_pi0_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print('N->pi^0 nu_alpha done...')

#for N->eta nu_alpha
pid_start="111"
pid_end="331"
start_mass=self.masses(pid_start)
end_mass=self.masses(pid_end)
pid1="221"
br_eta_nu=f"(coupling**2/(32*np.pi))*GF**2*hnl.fH(pid1)**2*mass**3*(1-self.masses(pid1)**2/mass**2)**2"
mass_produced=self.masses(pid1)
for n in range(len(nu_alpha)):
    pid2=nu_alpha[n]
    write_m_br_file(br_eta_nu,mass_produced,filename=f"model/width_temp/width_2body_ex_eta_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print('N->eta nu_alpha done...')

#for N->eta' nu_alpha
pid1="331"
br_etap_nu=f"(coupling**2/(32*np.pi))*GF**2*hnl.fH(pid1)**2*mass**3*(1-self.masses(pid1)**2/mass**2)**2"
mass_produced=self.masses(pid1)
for n in range(len(nu_alpha)):
    pid2=nu_alpha[n]
    write_m_br_file(br_etap_nu,mass_produced,filename=f"model/width_temp/width_2body_ex_etap_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print("N->eta' nu_alpha done...")

#for N->rho+ l_alpha-
grho=0.102
pid1="213"
br_rho_l=f"(coupling**2/(8*np.pi))*(grho**2/self.masses(pid1)**2)*GF**2*hnl.VH(pid1)**2*mass**3*((1-self.masses(pid2)**2/mass**2)**2+(self.masses(pid1)**2/mass**2)*(1+(self.masses(pid2)**2-2*self.masses(pid1)**2)/mass**2))*np.sqrt((1-(self.masses(pid1)-self.masses(pid2))**2/mass**2)*(1-(self.masses(pid1)+self.masses(pid2))**2/mass**2))"
for n in range(len(leptons)):
    pid2=leptons[n]
    mass_produced=self.masses(pid1)+self.masses(pid2)
    write_m_br_file(br_rho_l,mass_produced,filename=f"model/width_temp/width_2body_ex_rho_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
#charge conjugate processes
pid1="-213"
for n in range(len(leptons)):
    pid2=leptons[n]
    write_m_br_file(br_rho_l,mass_produced,filename=f"model/width_temp/width_2body_ex_rho_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print('N->rho+ l_alpha- done...')

#for N->rho0 nu_alpha
pid1="113"
pid2="11"
br_rho0_nu=f"(coupling**2/(16*np.pi))*(grho**2/self.masses(pid1)**2)*GF**2*mass**3*(1+2*self.masses(pid1)**2/mass**2)*(1-self.masses(pid1)**2/mass**2)**2"
mass_produced=self.masses(pid1)
for n in range(len(nu_alpha)):
    pid2=nu_alpha[n]
    write_m_br_file(br_rho0_nu,mass_produced,filename=f"model/width_temp/width_2body_ex_rho0_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print('N->rho0 nu_alpha done...')

#mass=1

#pseudoscalar and vector decays
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
#for N->P+ l- 
mass=1
charged_pseudo_symb=["K+","K-","D^+","D^-" ,"B^+" ,"B^-" ,"B^+_c","B_c^-"]
charged_pseudo=["321","-321","411","-411","521" ,"-521","541"  ,"-541"]
pid1="321"
pid2="11"
FP=lambda x,y: np.sqrt(lamda(1,x**2,y**2))*((1+x**2)*(1+x**2-y**2)-4*x**2)
yP=f"(self.masses(pid1)/mass)"
yl=f"(self.masses(pid2)/mass)"
coupling=f"hnl.vcoupling[str(abs(int(pid2)))]"
br_P_l=f"({coupling}**2*GF**2*mass**3*hnl.fH(pid1)**2*hnl.VH(pid1)**2/(16*np.pi))*{FP(eval(yl),eval(yP))}"
for n in range(len(charged_pseudo)):
    pid1=charged_pseudo[n]
    for m in range(len(leptons)):
        if int(pid1)<0:
            pid2="-"+leptons[m]
        else:
            pid2=leptons[m]
        mass_produced=self.masses(pid1)+self.masses(pid2)
        write_m_br_file(br_P_l,mass_produced,filename=f"model/width_temp/width_2body_ex_P_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print("N->P+ l- done...")

#for N->P0 nu
neutral_pseudo_symb=["K0","\bar{K}0","D^0","B^0","B_s^0"]
neutral_pseudo=["311","-311","421","511","531"]
#pid1="311"
#pid2="12"
coupling=f"hnl.vcoupling[str(abs(int(pid2))-1)]"
br_P0_nu=f"({coupling}**2*GF**2*mass**3*hnl.fH(pid1)**2/(64*np.pi))*(1-{yP}**2)**2"
for n in range(len(neutral_pseudo)):
    pid1=neutral_pseudo[n]
    for m in range(len(nu_alpha)):
        if int(pid1)<0:
            pid2="-"+nu_alpha[m]
        else:
            pid2=nu_alpha[m]
        mass_produced=self.masses(pid1)+self.masses(pid2)
        write_m_br_file(br_P0_nu,mass_produced,filename=f"model/width_temp/width_2body_ex_P0_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print("N->P0 nu done...")

#for N->V+ l-
charged_vec_symb=["K*+","K*-",]
charged_vec=["323","-323"]
#pid1="323"
#pid2="11"
FV=lambda x,y: np.sqrt(lamda(1,x**2,y**2))*((1-x**2)**2+(1+x**2)*y**2-2*y**4)
#it was self.masses(pid1)
yV=f"(self.masses(pid1)/mass)"
coupling=f"hnl.vcoupling[str(abs(int(pid2)))]"
br_V_l=f"({coupling}**2*GF**2*mass**3*hnl.fH(pid1)**2*hnl.VH(pid1)**2/(16*np.pi))*{FV(eval(yl),eval(yV))}"
for n in range(len(charged_vec)):
    pid1=charged_vec[n]
    for m in range(len(leptons)):
        if int(pid1)<0:
            pid2="-"+leptons[m]
        else:
            pid2=leptons[m]
        mass_produced=self.masses(pid1)+self.masses(pid2)
        write_m_br_file(br_V_l,mass_produced,filename=f"model/width_temp/width_2body_ex_V_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print("N->V+ l- done...")

#for N->V0 nu
neutral_vec_symb=["K*0","\bar{K}*0","\omega","D^{*0}","\bar{D}^{*0}"]
neutral_vec=["313","-313","223","423"   ,"-423"]
#pid1="313"
#pid2="12"
xw=f"0.231"
coupling=f"hnl.vcoupling[str(abs(int(pid2))-1)]"
yV=f"(self.masses(pid1)/mass)"
kv=1 #just a dummy value to get code to work
br_V0_nu=f"({coupling}**2*GF**2*mass**3*hnl.fH(pid1)**2*{kv}**2/(2*np.pi))*(1-{yV}**2)**2*(1+2*{yV}**2)"
for n in range(len(neutral_vec)):
    pid1=neutral_vec[n]
    if abs(int(pid1))==313:
        kv=f"(-1/4)+xw/3"
    if abs(int(pid1))==223:
        kv=f"xw/3"
    for m in range(len(nu_alpha)):
        if int(pid1)<0:
            pid2="-"+nu_alpha[m]
        else:
            pid2=nu_alpha[m]
        mass_produced=self.masses(pid1)+self.masses(pid2)
        write_m_br_file(br_V0_nu,mass_produced,filename=f"model/width_temp/width_2body_ex_V0_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass)
print("N->V0 nu done...")
convert_width_to_br_temp(path="model/width_temp/")

In [ ]:
#this part analyzes how well exclusive and inclusive decays into hadrons (widths) agree.
from matplotlib import pyplot as plt
br_or_width="width"
#br_or_width="width"
folder=f"width"
def add_br_files(files,br_or_width):
    folder=f"{br_or_width}"
    br_files=os.listdir(f"model/{folder}/")
    counter=0
    for n in range(len(br_files)):
        for m in range(len(files)):
            if br_files[n][0:len(files[m])] in files:
                counter+=1
                file_name=br_files[n]
                m_width_df_temp = pd.read_csv(f"model/{folder}/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False,dtype='float64')
                m_width_df_temp['width'] = m_width_df_temp['width'].fillna(0)
                if counter==1:
                    width=np.array(m_width_df_temp['width'].tolist())
                if counter!=1:
                    width_temp=np.array(m_width_df_temp['width'].tolist())
                    width+= width_temp
                    for l in range(len(width_temp)):
                        if width_temp[l]<0:
                            print(file_name)
                            pass
    mass=np.array(m_width_df_temp['mass'].tolist())
    return(mass,width)

#takes in a file and gives the x and y value of the file
def m_width(files,br_or_width):
    folder=f"{br_or_width}"
    m_width_df_temp = pd.read_csv(f"model/{folder}/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False,dtype='float64')
    width=np.array(m_width_df_temp['width'].tolist())
    mass=np.array(m_width_df_temp['mass'].tolist())
    return(mass,width)


#hadron curve inclusive
br_files=os.listdir(f"model/{folder}/")

had_file_char=f"{br_or_width}_3body_inc"
had_file_char1=f"{br_or_width}_2body_ex"
files_had=[f"{br_or_width}_3body_inc"]
mhad_inc,brhad_inc=add_br_files(files_had,br_or_width)

#hadron curve exclusive
files_had=[f"{br_or_width}_2body_ex"]
mhad_ex,brhad_ex=add_br_files(files_had,br_or_width)

plt.plot(mhad_inc,brhad_inc,label=r"hadrons (inc)")
plt.plot(mhad_ex,brhad_ex,label=r"hadrons (ex)")
plt.legend()
plt.xlabel(r"$M_N$")
plt.ylabel(r"$\Gamma$ (GeV)")
#plt.xlim(.1,10)
#plt.ylim(10**-22,10**-7)
plt.xscale("log")
plt.yscale("log")
plt.show()

#similar plot but dividing gamma by mN^5 as suggested by Felix
plt.plot(mhad_inc,brhad_inc/mhad_inc**5,label=r"hadrons (inc)")
plt.plot(mhad_ex,brhad_ex/mhad_ex**5,label=r"hadrons (ex)")
plt.legend()
plt.xlabel(r"$M_N$")
plt.ylabel(r"$\frac{\Gamma}{m_N^5}$")
plt.xlim(.1,10)
#plt.ylim(10**-14,10**-12)
plt.xscale("log")
plt.yscale("log")
plt.show()
print(brhad_ex.tolist())

In [ ]:
#comparing my ctau file with felix
from matplotlib import pyplot as plt

ctau_df = pd.read_csv(f"model/ctau.txt",delimiter='       ',engine='python',names=['mass','ctau'],index_col=False)
massf,ctauf=read_digitized_file(filename="/Users/alechewitt/Desktop/ctau_felix.csv",name_col1="x",name_col2="y")
mass=ctau_df['mass'].tolist()
ctau=ctau_df['ctau'].tolist()
plt.plot(mass,ctau,label="Alec")
plt.plot(massf,ctauf,label="Felix")
plt.legend()
plt.xlabel(r"$m_N (GeV)$")
plt.ylabel(r"$c \tau_N$")
plt.xscale("log")
plt.yscale("log")
plt.show()

In [ ]:
#list of mesons which the HNL can decay to.
#before I cleaned it up to separate functions from the writing files part
#only 3 body decays suggested by Jonathan


#removes br files, since this part will create them again
path=f"{src_path}/Models/HNL/model/br/*"
remove_files_from_folder(path)

#remove width files
path=f"{src_path}/Models/HNL/model/width/*"
remove_files_from_folder(path)

#remove ctau file
path=f"{src_path}/Models/HNL/model/ctau.txt"
if exists(path)==True:
    os.remove(f"{src_path}/Models/HNL/model/ctau.txt")


#self=model
#coupling=1
#GF=1.166378*10**(-5)

'''pseudoscalars_symbols=["K^0","\bar{K}^0","K^+","K^-" ,"\pi^0","\pi^+","\pi^-","\eta","\eta'","D_s^+","D_s^-","D0" ,"D^+","D^-" ,"B^0", "B^+","B^-" ]
pseudoscalars=        ["311","-311"     ,"321","-321","111"  ,"211"  ,"-211"  ,"221","331"  ,"431"  ,"-431" ,"421","411","-411","511","521" ,"-521"]
pseudoscalars_charged_symbols=["K^+","K^-" ,"\pi^+","\pi^-","D_s^+","D_s^-","D^+","D^-" ,"B^+" ,"B^-"  ]
pseudoscalars_charged=        ["321","-321","211"  ,"-211" ,"431"  ,"-431" ,"411","-411","521" ,"-521" ]
vectors_symbols=["K^{*0}","\bar{K}^{*0}","K^{*+}","K^{*-}","\rho_0","\rho^+","\rho^-","\omega","\phi","D^*0" ]
vectors=        ["313"   ,"-313"        ,"323"   ,"-323"  ,"113"   ,"213"   ,"-213"  ,"223"   ,"333" ,"423"  ]
vectors_charged_symbols=["K^{*+}","K^{*-}"]
vectors_charged=        ["323"   ,"-323"  ]''' #took out omega and phi, not sure where to find data, their lifetimes are super small 10^-23 roughly and they decay into channels we have implemented
'''leptons=["11","13","15"]
nu_alpha=["12","14","16"]'''



#GF=1.166378*10**(-5)
#leptons=["11","13","15"]
#nu_alpha=["12","14","16"]
#pid_start="111"
#pid_end="331"
#self=model
#start_mass=self.masses(pid_start)
#end_mass=self.masses(pid_end)
#end_mass=10
#end_mass=1.5


#for N->sum_{alpha beta} nu_alpha bar{nu}_beta nu_beta
pid1,pid2,pid3=["12","14","16"]
mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
#multiply by 2 to account for charge conjugate process
br_nu_bnu_nu="2*"+f"(GF**2*mass**5/(192*np.pi**3))*(hnl.vcoupling[str(abs(int(pid1))-1)]**2+hnl.vcoupling[str(abs(int(pid2))-1)]**2+hnl.vcoupling[str(abs(int(pid3))-1)]**2)"
write_m_br_file(br_nu_bnu_nu,mass_produced,filename=f"model/width/width_3body_lep_sum_nua_bnub_nub.txt")
print('3 body N->sum_{alpha beta} nu_alpha bar{nu}_beta nu_beta done...')

#for N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta
'''lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
I1_integrand=lambda s,x,y,z: (12/s)*(s-x**2-y**2)*(1+z**2-s)*np.sqrt(lamda(s,x**2,y**2))*np.sqrt(lamda(1,s,z**2))
I2_integrand=lambda s,x,y,z: (24*y*z/s)*(1+x**2-s)*np.sqrt(lamda(s,y**2,z**2))*np.sqrt(lamda(1,s,x**2))
delta=lambda l1,l2: 1 if l1==l2 else 0'''

'''def calc_br_la_lb_nub(pid1,pid2,pid3):
    yl1=f"(self.masses(pid1)/mass)"
    yl2=f"(self.masses(pid2)/mass)"
    ynul2=f"(self.masses(pid3)/mass)"
    x=yl1
    y=ynul2
    z=yl2
    I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
    br_la_lb_nub="2*"+f"(hnl.vcoupling[str(abs(int(pid1)))]**2*GF**2*mass**5/(192*np.pi**3))*{I1}*(1-delta(abs(int(pid1)),abs(int(pid2))))"
    #br_la_lb_nub=f"{delta(abs(int(pid1)),abs(int(pid2)))}"
    return(br_la_lb_nub)'''
for a in range(len(leptons)):
    for b in range(len(leptons)):
        if a!=b:
            pid1=leptons[a]
            pid2=leptons[b]
            pid2="-"+pid2
            pid3=nu_alpha[b]
            mass_produced=(self.masses(pid1)+self.masses(pid2)+self.masses(pid3))
            br_la_lb_nub=calc_br_la_lb_nub(pid1,pid2,pid3)
            write_m_br_file(br_la_lb_nub,mass_produced,filename=f"model/width/width_3body_lep_la_lb_nub_{pid1}_{pid2}_{pid3}.txt", write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
print('3 body N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta done...')

'''
#for N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
I1_integrand=lambda s,x,y,z: (12/s)*(s-x**2-y**2)*(1+z**2-s)*np.sqrt(lamda(s,x**2,y**2))*np.sqrt(lamda(1,s,z**2))
I2_integrand=lambda s,x,y,z: (24*y*z/s)*(1+x**2-s)*np.sqrt(lamda(s,y**2,z**2))*np.sqrt(lamda(1,s,x**2))
delta=lambda l1,l2: 1 if l1==l2 else 0
yl1=f"(self.masses(pid1)/mass)"
yl2=f"(self.masses(pid2)/mass)"
ynul2=f"(self.masses(pid3)/mass)"
x=yl1
y=ynul2
z=yl2
I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
br_la_lb_nub="2*"+f"(hnl.vcoupling[str(abs(int(pid1)))]**2*GF**2*mass**5/(192*np.pi**3))*{I1}*(1-delta(abs(int(pid1)),abs(int(pid2))))"
for a in range(len(leptons)):
    for b in range(len(leptons)):
        if a!=b:
            pid1=leptons[a]
            pid2=leptons[b]
            pid2="-"+pid2
            pid3=nu_alpha[b]
            mass_produced=(self.masses(pid1)+self.masses(pid2)+self.masses(pid3))
            write_m_br_file(br_la_lb_nub,mass_produced,filename=f"model/width/width_3body_lep_la_lb_nub_{pid1}_{pid2}_{pid3}.txt", write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
print('3 body N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta done...')
'''



#for N->l_beta^+ l_beta^- nu_alpha using daniels formula
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
I1_integrand=lambda s,x,y,z: (12/s)*(s-x**2-y**2)*(1+z**2-s)*np.sqrt(lamda(s,x**2,y**2))*np.sqrt(lamda(1,s,z**2))
I2_integrand=lambda s,x,y,z: (24*y*z/s)*(1+x**2-s)*np.sqrt(lamda(s,y**2,z**2))*np.sqrt(lamda(1,s,x**2))
#from scipy.integrate import quad
def calc_br_lb_lb_nua(pid1,pid2,pid3):
    delta=lambda l1,l2: 1 if l1==l2 else 0
    if pid2=="11":
        l2=1
    if pid2=="13":
        l2=2
    if pid2=="15":
        l2=3
    if pid3=="12":
        l1=1
    if pid3=="14":
        l1=2
    if pid3=="16":
        l1=3
    gL=f"(-(1/2)+xw)"
    gR=f"xw"
    x=f"0"
    y=f"(self.masses(pid2)/mass)"
    z=y
    I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
    I2=f"quad(I2_integrand,({y}+{z})**2,(1-{x})**2,args=({x},{y},{z}))[0]"
    coupling=f"hnl.vcoupling[str(abs(int(pid3))-1)]"
    br_lb_lb_nua="2*"+f"(coupling**2*GF**2*mass**5/(192*np.pi**3))*(({gL}*{gR}+{delta(l1,l2)}*{gR})*{I2}+({gL}**2+{gR}**2+{delta(l1,l2)}*(1+2*{gL}))*{I1})"
    return(br_lb_lb_nua)
for b in range(len(leptons)):
    for a in range(len(nu_alpha)):
        pid1=leptons[b]
        pid1="-"+pid1
        pid2=leptons[b]
        pid3=nu_alpha[a]
        mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
        br_lb_lb_nua=calc_br_lb_lb_nua(pid1,pid2,pid3)
        write_m_br_file(br_lb_lb_nua,mass_produced,filename=f"model/width/width_3body_lep_lb_lb_nua_{pid1}_{pid2}_{pid3}.txt", write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
print('3 body N->l_beta^+ l_beta^- nu_alpha done...')



#HNL decays 3 body (quarks)
#for N->U bar{D} l_alpha^-
#pid1=U; pid2=bar{D}; pid3=l_alpha^-
#from scipy.integrate import quad
#pid1="2"
#pid2="1"
#pid3="11"
#quarks_u_symbols=["u","c","t"]
#quarks_u=        ["2","4"]
#quarks_d_symbols=["d","s","b"]      #skipped top quark since it is very heavy
#quarks_d=        ["1","3","5"]
##pid1="211"
#br_H_l=f"(coupling**2/(16*np.pi))*GF**2*hnl.VH(pid1)**2*hnl.fH(pid1)**2*mass**3*((1-self.masses(pid2)**2/mass**2)**2-(self.masses(pid1)**2/mass**2)*(1+self.masses(pid2)**2/mass**2))*np.sqrt((1-(self.masses(pid1)-self.masses(pid2))**2/mass**2)*(1-(self.masses(pid1)+self.masses(pid2))**2/mass**2))"
'''I_integrand=lambda s,x,y,z: (12/s)*(s-x**2-y**2)*(1+z**2-s)*np.sqrt(lamda(s,x**2,y**2))*np.sqrt(lamda(1,s,z**2))
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
def calc_br_u_bd_l(pid1,pid2,pid3):
    xd=f"(self.masses(pid2)/mass)"
    xu=f"(self.masses(pid1)/mass)"
    xl=f"(self.masses(pid3)/mass)"
    x=xl
    y=xu
    z=xd
    I=f"quad(I_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
    coupling=f"hnl.vcoupling[str(abs(int(pid3)))]"
    #this should be multiplied by 2 
    br_u_d_l="2*"+f"(hnl.VHHp(pid1,pid2)**2*GF**2*mass**5*{coupling}**2/(64*np.pi**3))*{I}"
    return(br_u_d_l)'''

for u in range(len(quarks_u)):
    for d in range(len(quarks_d)):
        for a in range(len(leptons)):
            pid1=quarks_u[u]
            pid2=quarks_d[d]
            pid3=leptons[a]
            mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
            br_u_d_l=calc_br_u_bd_l(pid1,pid2,pid3)
            write_m_br_file(br_u_d_l,mass_produced,filename=f"model/width/width_3body_inc_u_d_l_{pid1}_{pid2}_{pid3}.txt",start_mass=end_mass,end_mass=10, write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
print('3 body N->U bar{D} l_alpha^- done...')

'''#HNL decays 3 body (quarks)
#inclusive decays
#for N-> q \bar{q} nu_l
nu_alpha=["12","14","16"]
GF=1.166378*10**(-5)
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
x=0
xq=f"(self.masses(pid1)/mass)"
y=xq
z=xq
xw=f"(0.231)" #xw=sin(theta_w)^2
I1_integrand=lambda s,x,y,z: (12/s)*(s-x**2-y**2)*(1+z**2-s)*np.sqrt(lamda(s,x**2,y**2))*np.sqrt(lamda(1,s,z**2))
I2_integrand=lambda s,x,y,z: (24*y*z/s)*(1+x**2-s)*np.sqrt(lamda(s,y**2,z**2))*np.sqrt(lamda(1,s,x**2))

def calc_br_q_bq_nu(pid1,pid3):
    if pid1 in quarks_u:
        gL=f"(1/2-(2/3)*{xw})"
        gR=f"(-(2/3)*{xw})" #article had 2 g_R^U so I assumed one was supposed to have a D
    if pid1 in quarks_d:
        gL=f"(-1/2+(1/3)*{xw})"
        gR=f"((1/3)*{xw})"
    I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
    I2=f"quad(I2_integrand,({y}+{z})**2,(1-{x})**2,args=({x},{y},{z}))[0]"
    coupling=f"hnl.vcoupling[str(abs(int(pid3))-1)]"
    #no factor of 2 since the charge conjugate process is the same process (I think the source already included factor of 2 here)
    #was a factor of 1/2 here
    br_q_bq_nu=f"({coupling}**2*GF**2*mass**5/(32*np.pi**3))*({gL}*{gR}*{I2}+({gL}**2+{gR}**2)*{I1})"
    return(br_q_bq_nu)'''

#pid1="111"
#br_pi0_nu=f"(coupling**2/(32*np.pi))*GF**2*hnl.fH(pid1)**2*mass**3*(1-self.masses(pid1)**2/mass**2)**2"

for q in range(len(quarks_u+quarks_d)):
    pid1=(quarks_u+quarks_d)[q]
    pid2="-"+pid1
    for a in range(len(nu_alpha)):
        pid3=nu_alpha[a]
        mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
        br_q_bq_nu=calc_br_q_bq_nu(pid1,pid3)
        write_m_br_file(br_q_bq_nu,mass_produced,filename=f"model/width/width_3body_inc_u_bu_l_{pid1}_{pid2}_{pid3}.txt",start_mass=start_mass,end_mass=10, write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
print('3 body N-> q \bar{q} nu_alpha done...')













#mass=1
'''
#pseudoscalar and vector decays
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
#for N->P+ l- 
#charged_pseudo_symb=["K+","pi+","D+"]
#charged_pseudo=["321","211","411"]
#pid1="321"
#pid2="11"
FP=lambda x,y: f"np.sqrt(lamda(1,{x}**2,{y}**2))*((1+{x}**2)*(1+{x}**2-{y}**2)-4*{x}**2)"
yP=f"(self.masses(pid1)/mass)"
yl=f"(self.masses(pid2)/mass)"
coupling=f"hnl.vcoupling[str(abs(int(pid2)))]"
#no factor of 2 added (probably should add)
br_P_l="2*"+f"({coupling}**2*GF**2*mass**3*hnl.fH(pid1)**2*hnl.VH(pid1)**2/(16*np.pi))*{FP(yl,yP)}"
for n in range(len(charged_pseudo)):
    pid1=charged_pseudo[n]
    for m in range(len(leptons)):
        if int(pid1)<0:
            pid2="-"+leptons[m]
        else:
            pid2=leptons[m]
        mass_produced=self.masses(pid1)+self.masses(pid2)
        write_m_br_file(br_P_l,mass_produced,filename=f"model/width/width_2body_ex_P_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)
print("N->P+ l- done...")'''



#pseudoscalar and vector decays
'''#for N->P+ l- 
#charged_pseudo_symb=["K+","pi+","D+"]
#charged_pseudo=["321","211","411"]
#pid1="321"
#pid2="11"
lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
def calc_br_P_l(pid1,pid2):
    FP=lambda x,y: f"np.sqrt(lamda(1,{x}**2,{y}**2))*((1+{x}**2)*(1+{x}**2-{y}**2)-4*{x}**2)"
    yP=f"(self.masses(pid1)/mass)"
    yl=f"(self.masses(pid2)/mass)"
    coupling=f"hnl.vcoupling[str(abs(int(pid2)))]"
    #no factor of 2 added (probably should add)
    br_P_l="2*"+f"({coupling}**2*GF**2*mass**3*hnl.fH(pid1)**2*hnl.VH(pid1)**2/(16*np.pi))*{FP(yl,yP)}"
    return(br_P_l)'''

for n in range(len(charged_pseudo)):
    pid1=charged_pseudo[n]
    for m in range(len(leptons)):
        if int(pid1)<0:
            pid2="-"+leptons[m]
        else:
            pid2=leptons[m]
        br_P_l=calc_br_P_l(pid1,pid2)
        mass_produced=self.masses(pid1)+self.masses(pid2)
        write_m_br_file(br_P_l,mass_produced,filename=f"model/width/width_2body_ex_P_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)
print("N->P+ l- done...")









#for N->P0 nu
#neutral_pseudo_symb=["K0" ,"\eta","\eta'","pi0","D0"]
#neutral_pseudo=     ["311","221" ,"331"  ,"111"]#,"421"]
#pid1="311"
#pid2="12"
coupling=f"hnl.vcoupling[str(abs(int(pid2))-1)]"
#no factor of 2 added for conjugate process, one has already been added
br_P0_nu=f"({coupling}**2*GF**2*mass**3*(hnl.fH(pid1))**2/(64*np.pi))*(1-{yP}**2)**2"
for n in range(len(neutral_pseudo)):
    pid1=neutral_pseudo[n]
    for m in range(len(nu_alpha)):
        if int(pid1)<0:
            pid2="-"+nu_alpha[m]
        else:
            pid2=nu_alpha[m]
        mass_produced=self.masses(pid1)+self.masses(pid2)
        write_m_br_file(br_P0_nu,mass_produced,filename=f"model/width/width_2body_ex_P0_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)
print("N->P0 nu done...")

#for N->V+ l-
'''#charged_vec_symb=["K*+","rho+","D*+","D*s+"]
#charged_vec=["323","213"]#,"413","433"]
#pid1="323"
#pid2="11"
def calc_br_V_l(pid1,pid2): 
    FV=lambda x,y: f"np.sqrt(lamda(1,{x}**2,{y}**2))*((1-{x}**2)**2+(1+{x}**2)*{y}**2-2*{y}**4)"
    #it was self.masses(pid1)
    yV=f"(self.masses(pid1)/mass)"
    yl=f"(self.masses(pid2)/mass)"
    coupling=f"hnl.vcoupling[str(abs(int(pid2)))]"
    #no factor of 2 added, should probably add it though
    br_V_l="2*"+f"({coupling}**2*GF**2*mass**3*hnl.fH(pid1)**2*hnl.VH(pid1)**2/(16*np.pi))*{FV(yl,yV)}"
    return(br_V_l)'''
for n in range(len(charged_vec)):
    pid1=charged_vec[n]
    for m in range(len(leptons)):
        if int(pid1)<0:
            pid2="-"+leptons[m]
        else:
            pid2=leptons[m]
        mass_produced=self.masses(pid1)+self.masses(pid2)
        br_V_l=calc_br_V_l(pid1,pid2)
        write_m_br_file(br_V_l,mass_produced,filename=f"model/width/width_2body_ex_V_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)
print("N->V+ l- done...")



#for N->V0 nu
'''#neutral_vec_symb=["K*0","\omega","\rho0","D*0"]
#neutral_vec=["313","223","113"]#,"423"]
#pid1="313"
#pid2="12"
def calc_br_V0_nu(pid1):
    xw=f"(0.231)"
    coupling=f"hnl.vcoupling[str(abs(int(pid2))-1)]"
    yV=f"(self.masses(pid1)/mass)"
    if abs(int(pid1))==313 or abs(int(pid1)==333):
        kv=f"((-1/4)+{xw}/3)"
    if abs(int(pid1))==223 or abs(int(pid1))==113:
        kv=f"({xw}/3)"
    if abs(int(pid1))==423 or abs(int(pid1))==443:
        kv=f"((1/4)-2*{xw}/3)"
    #no need to multiply by 2 for conjugate process, since it is already accounted for
    br_V0_nu=f"({coupling}**2*GF**2*mass**3*hnl.fH(pid1)**2*{kv}**2/(2*np.pi))*(1-{yV}**2)**2*(1+2*{yV}**2)"
    return(br_V0_nu)'''

for n in range(len(neutral_vec)):
    pid1=neutral_vec[n]
    br_V0_nu=calc_br_V0_nu(pid1)
    for m in range(len(nu_alpha)):
        if int(pid1)<0:
            pid2="-"+nu_alpha[m]
        else:
            pid2=nu_alpha[m]
        mass_produced=self.masses(pid1)+self.masses(pid2)
        write_m_br_file(br_V0_nu,mass_produced,filename=f"model/width/width_2body_ex_V0_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)

print("N->V0 nu done...")
write_m_ctau_file(filename="model/ctau.txt")
print("ctau.txt created...")
convert_width_to_br()
print("br files created...")

In [ ]:
#plot total hadronic width (exclusive) and total hadronic width (inclusive) and all contributions to exclusive
#before editing to split it into channels

#mbr_list is a list where each element has information to plot the decay channel
mbr_list=[]
br_or_width="width"
folder=f"{br_or_width}"
#pull all filenames in 'width' folder
br_files=os.listdir(f"model/{folder}/")
#len(br_files)
for n in range(0,len(br_files)):
    file_name=br_files[n]
    #print(file_name)
    if br_files[n][0:len("width_2body_ex")]=="width_2body_ex":
        m_width_df = pd.read_csv(f"model/width/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False)
        #extract pids from file
        pids=m_width_df['mass'][0].split("\t")
        #remove pid row
        m_width_df=m_width_df.iloc[1: , :]
        #extract m and width
        m=m_width_df['mass'].tolist()
        #convert mass list to float
        m=np.array([float(m[x]) for x in range(len(m))])
        brwidth=np.array(m_width_df['width'].tolist())
        #create label
        if pids[2]=='none':
            particle1=hnl.symbols(pids[0])
            particle2=hnl.symbols(pids[1])
            label=f"N -> {particle1} + {particle2}"
        if pids[2]!='none':
            particle1=hnl.symbols(pids[0])
            particle2=hnl.symbols(pids[1])
            particle3=hnl.symbols(pids[2])
            label=f"N -> {particle1} + {particle2} + {particle3}"
        #only plot if the widths are nonzero
        if sum(brwidth)!=0:
            mbr_list.append([m,brwidth/m**5,label,file_name,pids[0]])


#total decay to hadrons (inclusive)
br_files=os.listdir(f"model/{folder}/")
files_had=[f"{br_or_width}_3body_inc"]
mhad_inc,brhad_inc=add_br_files(files_had,br_or_width)
plt.plot(mhad_inc,brhad_inc/mhad_inc**5,label="Inclusive")

#total decay to hadrons (exclusive)
files_had=[f"{br_or_width}_2body_ex"]
mhad_ex,brhad_ex=add_br_files(files_had,br_or_width)
plt.plot(mhad_ex,brhad_ex/((mhad_ex**5)),label="Exclusive")

for n in range(len(mbr_list)):
    plt.plot(mbr_list[n][0],mbr_list[n][1],label=mbr_list[n][2])

plt.rcParams["figure.figsize"] = (10,10)
plt.legend(prop={'size':10})
plt.xlabel(r"$M_N$")
plt.ylabel(r"$\Gamma/m_N^5$ (GeV)")
plt.xlim(.1,10)
plt.ylim(10**-16,10**-12)
plt.xscale("log")
plt.yscale("log")
plt.show()

In [ ]:
#this was used when we were comparing daniels br curves with mine, it doesnt appear that we need these though.
#I dont think we need this
'''def remove_conjugate_file(br_files,br_files_alec):
    strings=["2body_lep","2body_tau","3body_H","3body_tau"]
    for l in range(len(strings)):
        if br_files[nn][0:len(strings[l])]==strings[l]:
            new_file=strings[l]
            count=0
            for m in range(len(br_files[nn])-3):
                if br_files[nn][m:m+3].isdigit() and count!=0:
                    #pid_new=str(-abs(int(br_files[nn][m:m+3])))
                    #new_file+=pid_new
                    if br_files[nn][m-1:m]=="-":
                        pid_new=str(-int(br_files[nn][m-1:m+3]))
                        new_file+="_"+pid_new
                    else:
                        pid_new=str(-int(br_files[nn][m:m+3]))
                        new_file+="_"+pid_new
                if br_files[nn][m:m+3].isdigit() and count==0:
                    if br_files[nn][m-1:m]=="-":
                        pid_new=str(int(br_files[nn][m-1:m+3]))
                        new_file+="_"+pid_new
                        count+=1
                    else:
                        pid_new=str(int(br_files[nn][m:m+3]))
                        new_file+="_"+pid_new
                        count+=1
                if br_files[nn][m:m+2].isdigit() and br_files[nn][m:m+3].isdigit()==False and br_files[nn][m-1:m+2].isdigit()==False and count!=0:
                    if br_files[nn][m-1:m]=="-":
                        pid_new=str(-int(br_files[nn][m-1:m+2]))
                        new_file+="_"+pid_new
                    else:
                        pid_new=str(-int(br_files[nn][m:m+2]))
                        new_file+="_"+pid_new
                if br_files[nn][m:m+2].isdigit() and br_files[nn][m:m+3].isdigit()==False and br_files[nn][m-1:m+2].isdigit()==False and count==0:
                    if br_files[nn][m-1:m]=="-":
                        pid_new=str(int(br_files[nn][m-1:m+2]))
                        new_file+="_"+pid_new
                        count+=1
                    else:
                        pid_new=str(int(br_files[nn][m:m+2]))
                        new_file+="_"+pid_new
                        count+=1
            new_file+=".txt"
    br_file_alec=new_file
    br_files_alec.remove(br_file_alec)'''

In [ ]:
#this was for plotting them, i dont think we need this though 
br_file="2body_lep_321_-15.txt"
df1=pd.read_csv(f"{src_path}/Models/HNL/model/BR_for_Alec/{br_file}",delimiter='\t',engine='python',names=['mass','br'],index_col=False,skiprows=4)
mass1=df1['mass'].tolist()
br1=df1['br'].tolist()
plt.clf()
plt.plot(mass1,br1,label="Daniel")
plt.xlim(0,.4)
#plt.ylim(0,1)
plt.legend()
plt.show()

In [ ]:
#stuff to implement br creation and tau creation and all the side files


    #helpful functions for creating files and reading files, etc.
    #takes a number and chops off part of it according to the precision, or delta
    def chop(self,expr, delta=10**-2):
        return np.ma.masked_inside(expr, -delta, delta).filled(0)
    #this portion will take a branching fraction and write to a file in the form m br
    #for the decays of HNL, takes a branching ratio and creates a file of mass br
    def write_m_br_file(self,br,mass_produced,filename="model/pi_nu.txt",start_mass=0,end_mass=10,write_pid=False,pid1_t=None,pid2_t=None,pid3_t=None):
        f = open(filename, "w+")
        f.close()
        f = open(filename, "a")
        mass=0.01
        delm=0.01
        if write_pid==True:
            if pid3_t!=None:
                f.write(pid1_t + "\t" + pid2_t + "\t" + pid3_t + "\n")
            if pid3_t==None:
                f.write(pid1_t + "\t" + pid2_t + "\t" + "none" + "\n")
        if write_pid==False:
            f.write("None" + "\n")
        steps=int((10-0)/0.01)-1
        for n in range(steps):
            if mass>mass_produced and mass>start_mass and mass<end_mass:
                f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(br)]))
            else:
                f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,0]))
            mass=chop(mass+delm)
        f.close()
        print(filename + ' created...')

    #same as write_m_br_file except for mass between, start_mass, end_mass is for the starting mass for br_had and ending mass for br_had
    def write_m_br_had_file(self,br,br_had,mass_produced,start_mass,end_mass,filename="model/pi_nu.txt"):
        f = open(filename, "w+")
        f.close()
        f = open(filename, "a")
        mass=start_mass
        delm=.01
        steps=int((end_mass-start_mass)/.01)+1
        for n in range(steps):
            if mass>mass_produced:
                f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(br_had)]))
            else:
                f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,0]))
            mass=chop(mass+delm)
        mass=end_mass
        delm=.01
        steps=int((10-end_mass)/.01)+1
        for n in range(steps):
            if mass>mass_produced:
                f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(br)]))
            else:
                f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,0]))

            mass=chop(mass+delm)
        f.close()
        print(filename + 'file created...')

    #this is for the decay N->nu_l1 l2 l2
    def write_m_brm_file(self,filename="model/pi_nu.txt"):
        f = open(filename, "w+")
        f.close()
        f = open(filename, "a")
        mass=.01
        br=bra_nul1_l2_l2(mass,l1,l2)+"*6.58*10**-14"
        delm=.01
        steps=int(10/.01)-1
        for n in range(steps):
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(br)]))
            mass=chop(mass+delm)
        f.close()
        print('file created...')

    def calc_m_tau(self,filename="model/tau_GeV.txt"):
        br_files=os.listdir("model/width/")
        lep_file_char="width_3body_lep"
        had_file_char="width_3body_inc"
        counter=0
        for n in range(len(br_files)):
            if br_files[n][0:len(lep_file_char)]==lep_file_char or br_files[n][0:len(had_file_char)]==had_file_char:
                counter+=1
                file_name=br_files[n]
                m_width_df_temp = pd.read_csv(f"model/width/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False)
                m_width_df_temp['width'] = m_width_df_temp['width'].fillna(0)
                if counter==1:
                    width=np.array(m_width_df_temp['width'].tolist())
                if counter!=1:
                    width_temp=np.array(m_width_df_temp['width'].tolist())
                    width+= width_temp
        mass=np.array(m_width_df_temp['mass'].tolist())
        tau=1/width
        return(mass,tau)

    def calc_m_ctau(self,filename="model/ctau.txt"):
        br_files=os.listdir("model/width/")
        lep_file_char="width_3body_lep"
        had_file_char="width_2body_ex"
        hadf_file_char="width_3body_inc"
        counter=0
        for n in range(len(br_files)):
            if br_files[n][0:len(lep_file_char)]==lep_file_char or br_files[n][0:len(had_file_char)]==had_file_char or br_files[n][0:len(hadf_file_char)]==hadf_file_char:
                counter+=1
                file_name=br_files[n]
                m_width_df_temp = pd.read_csv(f"model/width/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False)
                m_width_df_temp['width'] = m_width_df_temp['width'].fillna(0)
                if counter==1:
                    width=np.array(m_width_df_temp['width'].tolist())
                if counter!=1:
                    width_temp=np.array(m_width_df_temp['width'].tolist())
                    width+= width_temp
        mass=np.array(m_width_df_temp['mass'].tolist())
        ctau=((1/(width*10**-7))*((6.582122*pow(10.,-25.))))*3*10**8
        return(mass,ctau)

    def write_m_ctau_file(self,filename="model/ctau.txt"):
        br_files=os.listdir("model/width/")
        lep_file_char="width_3body_lep"
        had_file_char="width_2body_ex"
        hadf_file_char="width_3body_inc"
        counter=0
        for n in range(len(br_files)):
            if br_files[n][0:len(lep_file_char)]==lep_file_char or br_files[n][0:len(had_file_char)]==had_file_char or br_files[n][0:len(hadf_file_char)]==hadf_file_char:
                counter+=1
                file_name=br_files[n]
                m_width_df_temp = pd.read_csv(f"model/width/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False,skiprows=1)
                m_width_df_temp['width'] = m_width_df_temp['width'].fillna(0)
                if counter==1:
                    width=np.array(m_width_df_temp['width'].tolist())
                if counter!=1:
                    width_temp=np.array(m_width_df_temp['width'].tolist())
                    width+= width_temp
        mass=np.array(m_width_df_temp['mass'].tolist())
        ctau=((1/(width*10**-7))*((6.582122*pow(10.,-25.))))*3*10**8
        f = open(filename, "w+")
        f.close()
        f = open(filename, "a")
        if len(mass)!=len(ctau):
            print("the masses dont match the shape of ctau!!!")
        for n in range(len(mass)):
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass[n],ctau[n]]))
        f.close()

    def read_m_br(self,characteristic):
        br_files=os.listdir("model/width/")
        width=[]
        counter=0
        for n in range(len(br_files)):
            if br_files[n][0:len(characteristic)]==characteristic:
                counter+=1
                file_name=br_files[n]
                m_width_df_temp = pd.read_csv(f"model/width/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False)
                if counter==1:
                    width=np.array(m_width_df_temp['width'].tolist())
                if counter!=1:
                    width_temp=np.array(m_width_df_temp['width'].tolist())
                    width+= width_temp
        mass=np.array(m_width_df_temp['mass'].tolist())
        return (mass,width,counter)

    #"/Users/alechewitt/Desktop/felix_pts.csv"
    def read_digitized_file(self,filename="/Users/alechewitt/Desktop/felix_pts.csv",name_col1="x",name_col2="y"):
        x_y_df = pd.read_csv(filename,delimiter=',',engine='python',names=[name_col1,name_col2],index_col=False)
        x=np.array(x_y_df[name_col1].tolist())[4:]
        y=np.array(x_y_df[name_col2].tolist())[4:]
        x=[float(a) for a in x]
        y=[float(a) for a in y]
        return(x,y)

    def read_file(self,filename,name_col1="x",name_col2="y"):
        x_y_df = pd.read_csv(filename,delimiter='       ',engine='python',names=[name_col1,name_col2],index_col=False)
        x=np.array(x_y_df[name_col1].tolist())[4:]
        y=np.array(x_y_df[name_col2].tolist())[4:]
        return(x,y)

    def convert_width_to_br(self):
        br_files=os.listdir("model/width/")
        ctau=pd.read_csv(f"model/ctau.txt",delimiter='       ',engine='python',names=['mass','ctau'],index_col=False)
        for n in range(len(br_files)):
            file_name=br_files[n]
            m_width_df = pd.read_csv(f"model/width/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False,skiprows=1)
            m_width_df['width']=m_width_df['width']*ctau['ctau']*10**-7/(((6.582122*pow(10.,-25.)))*3*10**8)
            numpy_array = m_width_df.to_numpy()
            new_file_name="br"+br_files[n][len("width"):]
            np.savetxt(f"model/br/{new_file_name}", numpy_array,delimiter='       ')

    def convert_width_to_br_temp(self,path="model/width_temp/"):
        br_files=os.listdir(path)
        ctau=pd.read_csv(f"model/ctau.txt",delimiter='       ',engine='python',names=['mass','ctau'],index_col=False)
        for n in range(len(br_files)):
            file_name=br_files[n]
            m_width_df = pd.read_csv(path + f"{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False)
            m_width_df['width']=m_width_df['width']*ctau['ctau']*10**-7/(((6.582122*pow(10.,-25.)))*3*10**8)
            numpy_array = m_width_df.to_numpy()
            new_file_name="br"+br_files[n][len("width"):]
            np.savetxt(f"model/br_temp/{new_file_name}", numpy_array,delimiter='       ')



    #input path to given folder and it removes all files in that folder
    def remove_files_from_folder(self,path):
        files = glob.glob(path)
        for f in files:
            os.remove(f)


    def add_br_files(self,files,br_or_width):
        folder=f"{br_or_width}"
        br_files=os.listdir(f"model/{folder}/")
        counter=0
        for n in range(len(br_files)):
            for m in range(len(files)):
                if br_files[n][0:len(files[m])] in files:
                    counter+=1
                    file_name=br_files[n]
                    m_width_df_temp = pd.read_csv(f"model/{folder}/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False,dtype='float64',skiprows=1)
                    m_width_df_temp['width'] = m_width_df_temp['width'].fillna(0)
                    print("counter, should go up to 1 ",counter)
                    if counter==1:
                        width=np.array(m_width_df_temp['width'].tolist())
                    if counter!=1:
                        width_temp=np.array(m_width_df_temp['width'].tolist())
                        width+= width_temp
                        for l in range(len(width_temp)):
                            if width_temp[l]<0:
                                print(file_name)
                                pass
        mass=np.array(m_width_df_temp['mass'].tolist())
        return(mass,width)

    #for group of mesons, takes a list of file names and adds the widths
    def add_br_files_group(self,files,br_or_width):
        folder=f"{br_or_width}"
        counter=0
        for m in range(len(files)):
            file_name=files[m]+".txt"
            if counter==0:
                m_width_df = pd.read_csv(f"model/{folder}/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False,dtype='float64',skiprows=1)
                counter+=1
                width=np.array(m_width_df['width'].tolist())
                mass=np.array(m_width_df['mass'].tolist())


            if counter!=0:
                m_width_df_temp = pd.read_csv(f"model/{folder}/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False,dtype='float64',skiprows=1)
                width_temp=np.array(m_width_df_temp['width'].tolist())
                width+=width_temp
        return(mass,width)



    #list of particles for the decay channels, start and end masses

    #xw=0.231
    leptons=["11","13","15"]
    nu_alpha=["12","14","16"]
    '''    pid_start="111"
    pid_end="331"
    #self=model
    start_mass=self.masses(pid_start)
    end_mass=self.masses(pid_end)
    end_mass=10'''


    quarks_u_symbols=["u","c","t"]
    quarks_u=        ["2","4"]          #skipped top since it is heavy
    quarks_d_symbols=["d","s","b"]   
    quarks_d=        ["1","3","5"]


    charged_pseudo_symb=["K+" ,"pi+","D+" ,"D+s","B+" ,"Bc+"]
    charged_pseudo=     ["321","211","411","431","521","541"]


    neutral_pseudo_symb=["K0" ,"\eta","\eta'","pi0","D0" ,"B0" ,"B0s"]
    neutral_pseudo=     ["311","221" ,"331"  ,"111","421","511","531"]


    charged_vec_symb=["K*+","rho+","D*+","D*s+","B*+"]
    charged_vec=     ["323","213" ,"413","433" ,"523"]


    neutral_vec_symb=["K*0","\omega","\rho0","phi","J/psi","D*0"]#,"B*0","B*0s"]
    neutral_vec=     ["313","223"   ,"113"  ,"333","443"  ,"423"]#,"513","533" ]

    separate_channels=["1","2","3","4","5","113","213","223","333","443","221","331"]



    #functions to calculate decay widths for HNL
    #source for decay widths: https://arxiv.org/pdf/1005.1607.pdf

    #auxiliary functions 
    lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
    I1_integrand=lambda s,x,y,z: (12/s)*(s-x**2-y**2)*(1+z**2-s)*np.sqrt(lamda(s,x**2,y**2))*np.sqrt(lamda(1,s,z**2))
    I2_integrand=lambda s,x,y,z: (24*y*z/s)*(1+x**2-s)*np.sqrt(lamda(s,y**2,z**2))*np.sqrt(lamda(1,s,x**2))
    delta=lambda l1,l2: 1 if l1==l2 else 0
    xw=f"(0.231)" #xw=sin(theta_w)^2
    x=0
    xq=f"(self.masses(pid1)/mass)"
    y=xq
    z=xq

    #there is an extra coupling factor to allow all the actual couplings to get scaled if needed later
    coupling=1



    #for N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta
    def calc_br_la_lb_nub(self,pid1,pid2,pid3):
        yl1=f"(self.masses(pid1)/mass)"
        yl2=f"(self.masses(pid2)/mass)"
        ynul2=f"(self.masses(pid3)/mass)"
        x=yl1
        y=ynul2
        z=yl2
        I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
        br_la_lb_nub="2*"+f"(self.vcoupling[str(abs(int(pid1)))]**2*coupling**2*self.GF()**2*mass**5/(192*np.pi**3))*{I1}*(1-delta(abs(int(pid1)),abs(int(pid2))))"
        return(br_la_lb_nub)

    #for N->l_beta^+ l_beta^- nu_alpha 
    def calc_br_lb_lb_nua(self,pid1,pid2,pid3):
        delta=lambda l1,l2: 1 if l1==l2 else 0
        if pid2=="11":
            l2=1
        if pid2=="13":
            l2=2
        if pid2=="15":
            l2=3
        if pid3=="12":
            l1=1
        if pid3=="14":
            l1=2
        if pid3=="16":
            l1=3
        xw=f"(0.231)"

        gL=f"(-(1/2)+{xw})"
        gR=f"{xw}"
        x=f"0"
        y=f"(self.masses(pid2)/mass)"
        z=y
        I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
        I2=f"quad(I2_integrand,({y}+{z})**2,(1-{x})**2,args=({x},{y},{z}))[0]"
        coupling1=f"self.vcoupling[str(abs(int(pid3))-1)]"
        br_lb_lb_nua="2*"+f"({coupling1}**2*coupling**2*self.GF()**2*mass**5/(192*np.pi**3))*(({gL}*{gR}+{delta(l1,l2)}*{gR})*{I2}+({gL}**2+{gR}**2+{delta(l1,l2)}*(1+2*{gL}))*{I1})"
        return(br_lb_lb_nua)

    #for N->U bar{D} l_alpha^-
    def calc_br_u_bd_l(self,pid1,pid2,pid3):
        xd=f"(self.masses(pid2)/mass)"
        xu=f"(self.masses(pid1)/mass)"
        xl=f"(self.masses(pid3)/mass)"
        x=xl
        y=xu
        z=xd
        I=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
        coupling1=f"self.vcoupling[str(abs(int(pid3)))]"
        #this should be multiplied by 2 
        br_u_d_l="2*"+f"(self.VHHp(pid1,pid2)**2*self.GF()**2*mass**5*{coupling1}**2*coupling**2/(64*np.pi**3))*{I}"
        return(br_u_d_l)

    #HNL decays 3 body (quarks)
    #inclusive decays
    #for N-> q \bar{q} nu_l
    def calc_br_q_bq_nu(self,pid1,pid3):
        if pid1 in quarks_u:
            gL=f"(1/2-(2/3)*{xw})"
            gR=f"(-(2/3)*{xw})" #article had 2 g_R^U so I assumed one was supposed to have a D
        if pid1 in quarks_d:
            gL=f"(-1/2+(1/3)*{xw})"
            gR=f"((1/3)*{xw})"
        I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
        I2=f"quad(I2_integrand,({y}+{z})**2,(1-{x})**2,args=({x},{y},{z}))[0]"
        coupling1=f"self.vcoupling[str(abs(int(pid3))-1)]"
        #no factor of 2 since the charge conjugate process is the same process (I think the source already included factor of 2 here)
        #was a factor of 1/2 here
        br_q_bq_nu=f"({coupling1}**2*coupling**2*self.GF()**2*mass**5/(32*np.pi**3))*({gL}*{gR}*{I2}+({gL}**2+{gR}**2)*{I1})"
        return(br_q_bq_nu)

    #for N->P+ l- 
    def calc_br_P_l(self,pid1,pid2):
        FP=lambda x,y: f"np.sqrt(lamda(1,{x}**2,{y}**2))*((1+{x}**2)*(1+{x}**2-{y}**2)-4*{x}**2)"
        yP=f"(self.masses(pid1)/mass)"
        yl=f"(self.masses(pid2)/mass)"
        coupling1=f"self.vcoupling[str(abs(int(pid2)))]"
        #no factor of 2 added (probably should add)
        br_P_l="2*"+f"({coupling1}**2*coupling**2*self.GF()**2*mass**3*self.fH(pid1)**2*self.VH(pid1)**2/(16*np.pi))*{FP(yl,yP)}"
        return(br_P_l)

    def calc_br_P0_nu(self,pid1,pid2):
        coupling1=f"self.vcoupling[str(abs(int(pid2))-1)]"
        #no factor of 2 added for conjugate process, one has already been added
        yP=f"(self.masses(pid1)/mass)"
        br_P0_nu=f"({coupling1}**2*coupling**2*self.GF()**2*mass**3*(self.fH(pid1))**2/(64*np.pi))*(1-{yP}**2)**2"
        return(br_P0_nu)

    #for N->V+ l-
    def calc_br_V_l(self,pid1,pid2): 
        FV=lambda x,y: f"np.sqrt(lamda(1,{x}**2,{y}**2))*((1-{x}**2)**2+(1+{x}**2)*{y}**2-2*{y}**4)"
        #it was self.masses(pid1)
        yV=f"(self.masses(pid1)/mass)"
        yl=f"(self.masses(pid2)/mass)"
        coupling1=f"self.vcoupling[str(abs(int(pid2)))]"
        #no factor of 2 added, should probably add it though
        br_V_l="2*"+f"({coupling1}**2*coupling**2*self.GF()**2*mass**3*self.fH(pid1)**2*self.VH(pid1)**2/(16*np.pi))*{FV(yl,yV)}"
        return(br_V_l)

    #for N->V0 nu
    def calc_br_V0_nu(self,pid1):
        xw=f"(0.231)"
        coupling1=f"self.vcoupling[str(abs(int(pid2))-1)]"
        yV=f"(self.masses(pid1)/mass)"
        if abs(int(pid1))==313 or abs(int(pid1)==333):
            kv=f"((-1/4)+{xw}/3)"
        if abs(int(pid1))==223 or abs(int(pid1))==113:
            kv=f"({xw}/3)"
        if abs(int(pid1))==423 or abs(int(pid1))==443:
            kv=f"((1/4)-2*{xw}/3)"
        #no need to multiply by 2 for conjugate process, since it is already accounted for
        br_V0_nu=f"({coupling1}**2*coupling**2*self.GF()**2*mass**3*self.fH(pid1)**2*{kv}**2/(2*np.pi))*(1-{yV}**2)**2*(1+2*{yV}**2)"
        return(br_V0_nu)


    #list of mesons which the HNL can decay to. 
    #only 3 body decays suggested by Jonathan

    def get_br_and_ctau(self,src_path):
        #self.GF()=1.166378*10**(-5)

        pid_start="111"
        pid_end="331"
        #self=model
        start_mass=self.masses(pid_start)
        end_mass=self.masses(pid_end)
        end_mass=10

        #removes br files, since this part will create them again
        path=f"{src_path}/Models/HNL/model/br/*"
        path1=f"{src_path}/Models/HNL/model/br/"
        path1len=len(listdir(path1))
        if path1len!=0:
            self.remove_files_from_folder(path)
        #remove_files_from_folder(path)

        #remove width files, so it can update it
        path=f"{src_path}/Models/HNL/model/width/*"
        path1=f"{src_path}/Models/HNL/model/width/"
        path1len=len(listdir(path1))
        if path1len!=0:
            self.remove_files_from_folder(path)

        #remove ctau file, so it can update it
        path=f"{src_path}/Models/HNL/model/ctau.txt"
        if exists(path)==True:
            os.remove(f"{src_path}/Models/HNL/model/ctau.txt")


        ##########creates width files############

        #for N->sum_{alpha beta} nu_alpha bar{nu}_beta nu_beta
        pid1,pid2,pid3=["12","14","16"]
        mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
        #multiply by 2 to account for charge conjugate process
        br_nu_bnu_nu="2*"+f"(self.GF()**2*mass**5/(192*np.pi**3))*(self.vcoupling[str(abs(int(pid1))-1)]**2+self.vcoupling[str(abs(int(pid2))-1)]**2+self.vcoupling[str(abs(int(pid3))-1)]**2)"
        self.write_m_br_file(br_nu_bnu_nu,mass_produced,filename=f"model/width/width_3body_lep_sum_nua_bnub_nub.txt")
        print('3 body N->sum_{alpha beta} nu_alpha bar{nu}_beta nu_beta done...')

    '''
    #for N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta
    for a in range(len(leptons)):
        for b in range(len(leptons)):
            if a!=b:
                pid1=leptons[a]
                pid2=leptons[b]
                pid2="-"+pid2
                pid3=nu_alpha[b]
                mass_produced=(self.masses(pid1)+self.masses(pid2)+self.masses(pid3))
                br_la_lb_nub=calc_br_la_lb_nub(pid1,pid2,pid3)
                write_m_br_file(br_la_lb_nub,mass_produced,filename=f"model/width/width_3body_lep_la_lb_nub_{pid1}_{pid2}_{pid3}.txt", write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
    print('3 body N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta done...')


    #for N->l_beta^+ l_beta^- nu_alpha using daniels formula
    for b in range(len(leptons)):
        for a in range(len(nu_alpha)):
            pid1=leptons[b]
            pid1="-"+pid1
            pid2=leptons[b]
            pid3=nu_alpha[a]
            mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
            br_lb_lb_nua=calc_br_lb_lb_nua(pid1,pid2,pid3)
            write_m_br_file(br_lb_lb_nua,mass_produced,filename=f"model/width/width_3body_lep_lb_lb_nua_{pid1}_{pid2}_{pid3}.txt", write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
    print('3 body N->l_beta^+ l_beta^- nu_alpha done...')


    #HNL decays 3 body (quarks)
    #inclusive decays
    #for N->U bar{D} l_alpha^-'
    for u in range(len(quarks_u)):
        for d in range(len(quarks_d)):
            for a in range(len(leptons)):
                pid1=quarks_u[u]
                pid2=quarks_d[d]
                pid3=leptons[a]
                mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
                br_u_d_l=calc_br_u_bd_l(pid1,pid2,pid3)
                write_m_br_file(br_u_d_l,mass_produced,filename=f"model/width/width_3body_inc_u_d_l_{pid1}_{pid2}_{pid3}.txt",start_mass=start_mass,end_mass=10, write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
    print('3 body N->U bar{D} l_alpha^- done...')


    #for N-> q \bar{q} nu_l
    for q in range(len(quarks_u+quarks_d)):
        pid1=(quarks_u+quarks_d)[q]
        pid2="-"+pid1
        for a in range(len(nu_alpha)):
            pid3=nu_alpha[a]
            mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
            br_q_bq_nu=calc_br_q_bq_nu(pid1,pid3)
            write_m_br_file(br_q_bq_nu,mass_produced,filename=f"model/width/width_3body_inc_u_bu_l_{pid1}_{pid2}_{pid3}.txt",start_mass=start_mass,end_mass=10, write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
    print('3 body N-> q \bar{q} nu_alpha done...')


    #pseudoscalar and vector decays
    #for N-> P+ l-
    for n in range(len(charged_pseudo)):
        pid1=charged_pseudo[n]
        for m in range(len(leptons)):
            if int(pid1)<0:
                pid2="-"+leptons[m]
            else:
                pid2=leptons[m]
            br_P_l=calc_br_P_l(pid1,pid2)
            mass_produced=self.masses(pid1)+self.masses(pid2)
            write_m_br_file(br_P_l,mass_produced,filename=f"model/width/width_2body_ex_P_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)
    print("N->P+ l- done...")


    #for N->P0 nu
    for n in range(len(neutral_pseudo)):
        pid1=neutral_pseudo[n]
        for m in range(len(nu_alpha)):
            if int(pid1)<0:
                pid2="-"+nu_alpha[m]
            else:
                pid2=nu_alpha[m]
            mass_produced=self.masses(pid1)+self.masses(pid2)
            br_P0_nu=calc_br_P0_nu(pid1,pid2)
            write_m_br_file(br_P0_nu,mass_produced,filename=f"model/width/width_2body_ex_P0_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)
    print("N->P0 nu done...")


    #for N->V+ l-
    for n in range(len(charged_vec)):
        pid1=charged_vec[n]
        for m in range(len(leptons)):
            if int(pid1)<0:
                pid2="-"+leptons[m]
            else:
                pid2=leptons[m]
            mass_produced=self.masses(pid1)+self.masses(pid2)
            br_V_l=calc_br_V_l(pid1,pid2)
            write_m_br_file(br_V_l,mass_produced,filename=f"model/width/width_2body_ex_V_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)
    print("N->V+ l- done...")


    #for N->V0 nu
    for n in range(len(neutral_vec)):
        pid1=neutral_vec[n]
        br_V0_nu=calc_br_V0_nu(pid1)
        for m in range(len(nu_alpha)):
            if int(pid1)<0:
                pid2="-"+nu_alpha[m]
            else:
                pid2=nu_alpha[m]
            mass_produced=self.masses(pid1)+self.masses(pid2)
            write_m_br_file(br_V0_nu,mass_produced,filename=f"model/width/width_2body_ex_V0_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)


    print("N->V0 nu done...")
    write_m_ctau_file(filename="model/ctau.txt")
    print("ctau.txt created...")
    convert_width_to_br()
    print("br files created...")
    '''


